<a href="https://colab.research.google.com/github/ryanczhang7/spotifyproject/blob/master/Spotify_Data_Acquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting HipHop Song Data from Spotify Web API using Spotipy

**Note: hiphop data gathered on 3/1 and pop data gathered on 3/6.**

In [0]:
pip install spotipy

In [0]:
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import requests

**I thank RareLoot at Medium for helping me with some starter code for this process with his article here: https://medium.com/@RareLoot/extracting-spotify-data-on-your-favourite-artist-via-python-d58bc92a4330.**

In [0]:
# The spotipy starter code for accessing the API
import spotipy

client_id = 
client_secret = 
# To access authorised Spotify data
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id, client_secret=client_secret)
# Spotify object to access API
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [0]:
# Searching for an artist to get his uri, which I need to get his songs
name = "Kendrick Lamar"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/2YZyLoL8N0Wb9xBt1NhZWg'},
  'href': 'https://api.spotify.com/v1/artists/2YZyLoL8N0Wb9xBt1NhZWg',
  'id': '2YZyLoL8N0Wb9xBt1NhZWg',
  'name': 'Kendrick Lamar',
  'type': 'artist',
  'uri': 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]['uri']
# Pull all of the artist's albums
sp_albums = sp.artist_albums(artist_uri, album_type='album')
# Store artist's albums' names' and uris in separate lists
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
    
album_uris

['spotify:album:3pLdWdkj83EYfDN6H2N8MR',
 'spotify:album:4Kp52vGJe5oVi0EXNhSDKy',
 'spotify:album:5sOSzueqgCiVpXNcpd6QpL',
 'spotify:album:4alcGHjstaALJHHiljfy3H',
 'spotify:album:0bDN3eDKOfaeM5vccGu6SI',
 'spotify:album:1LLOGSa5ehwSWrPg1Qc3Vg',
 'spotify:album:4eLPsYPBmXABThSJ821sqY',
 'spotify:album:2ZsOtrjbcxMhcOcyDzNt1V',
 'spotify:album:0bLXUfNT34mna9aXq8ex68',
 'spotify:album:0kL3TYRsSXnu0iJvFO3rud',
 'spotify:album:7ycBtnsMtyVbbwTfJwRjSP',
 'spotify:album:76cek8jEdNsxH60aUu8Qtj',
 'spotify:album:6w7lqIsvDPgTChMrPw5oIL',
 'spotify:album:0CXmtbhJapIH7vw8go2DEb',
 'spotify:album:2dnvkiEsLtEMOAmM0DaZVo',
 'spotify:album:2ioBMJIiOANfeCaXP9a7Zn',
 'spotify:album:21EBAOQBAPvE7bMsz6sGV7',
 'spotify:album:748dZDqSZy6aPXKcI9H80u',
 'spotify:album:1DqhWr73Fh5yoNzKLas0G3',
 'spotify:album:1gItED20w7gVObqo6sSh8y']

In [0]:
album_names

['Black Panther The Album Music From And Inspired By',
 'Black Panther The Album Music From And Inspired By',
 'Black Panther The Album Music From And Inspired By',
 'DAMN. COLLECTORS EDITION.',
 'DAMN. COLLECTORS EDITION.',
 'DAMN. COLLECTORS EDITION.',
 'DAMN.',
 'DAMN.',
 'DAMN.',
 'untitled unmastered.',
 'To Pimp A Butterfly',
 'To Pimp A Butterfly',
 'To Pimp A Butterfly',
 'To Pimp A Butterfly',
 'good kid, m.A.A.d city (Deluxe)',
 'good kid, m.A.A.d city',
 'good kid, m.A.A.d city',
 'good kid, m.A.A.d city (Deluxe)',
 'good kid, m.A.A.d city (Deluxe)',
 'good kid, m.A.A.d city (Deluxe)']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

14
14
14
14
14
14
14
14
14
8
16
16
16
16
16
13
13
17
16
15


**There are many duplicates in the album_names, and when I pull up how many tracks there are in each, I see that they are duplicates. Only good kid, mAAd city has differences, so I will pick the one with the most songs. They ablums I want to get are at indices 0,3,9,10,17.**

In [0]:
# Removing duplicate albums from both lists at the same time
indices = [0,3,9,10,17]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Black Panther The Album Music From And Inspired By',
 'DAMN. COLLECTORS EDITION.',
 'untitled unmastered.',
 'To Pimp A Butterfly',
 'good kid, m.A.A.d city (Deluxe)']

In [0]:
# Testing getting all song in an album
for song in sp.album_tracks(new_album_uris[0])["items"]:
  print(song["name"])

Black Panther
All The Stars (with SZA)
X (with 2 Chainz & Saudi)
The Ways (with Swae Lee)
Opps (with Yugen Blakrok)
I Am
Paramedic!
Bloody Waters (with Anderson .Paak & James Blake)
King's Dead (with Kendrick Lamar, Future & James Blake)
Redemption Interlude
Redemption (with Babes Wodumo)
Seasons (with Sjava & Reason)
Big Shot (with Travis Scott)
Pray For Me (with Kendrick Lamar)


In [0]:
# Function to pull song data from API using an artist's album uri, for all
# the albums that I chose.
def albumSongs(uri, name, album):
  
  album_tracks = []
  
  # pull data on album tracks
  tracks = sp.album_tracks(uri) 

  # for each track, create dictionary for that specific tracks
  for song in tracks['items']:
    track = {} #
    track["artist"] = name
    track['album'] = album
    track['uri'] = song['uri']
    track['name'] = song['name']
    track['artists'] = []
    for artist in song['artists']:
      track['artists'].append(artist["name"])
    track['track_number'] = song['track_number']
    track['duration_ms'] = song['duration_ms']
    track['explicit'] = song['explicit']
    # append each track dictionary to the list
    album_tracks.append(track)

  return album_tracks

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 

Album Black Panther The Album Music From And Inspired By songs has been added to spotify_tracks
Album DAMN. COLLECTORS EDITION. songs has been added to spotify_tracks
Album untitled unmastered. songs has been added to spotify_tracks
Album To Pimp A Butterfly songs has been added to spotify_tracks
Album good kid, m.A.A.d city (Deluxe) songs has been added to spotify_tracks


In [0]:
df_kendrick = json_normalize(spotify_tracks)
df_kendrick

,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,spotify:track:5qN4HFkapdAOV94XPryVof,Black Panther,[Kendrick Lamar],1,130613,True
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,All The Stars (with SZA),"[Kendrick Lamar, SZA]",2,232186,True
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,spotify:track:4LmAnpjlhWTahvRkYR8xJa,X (with 2 Chainz & Saudi),"[ScHoolboy Q, 2 Chainz, Saudi]",3,267426,True
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,spotify:track:5jyyPsIGM2yqkZN9R3TmvN,The Ways (with Swae Lee),"[Khalid, Swae Lee]",4,238893,True
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,spotify:track:7bUcBztfGqO7cSI2gMZeCI,Opps (with Yugen Blakrok),"[Vince Staples, Yugen Blakrok]",5,180893,True
...,...,...,...,...,...,...,...,...
64,Kendrick Lamar,"good kid, m.A.A.d city (Deluxe)",spotify:track:5jHilrZDaP1vJMZfdN1He3,The Recipe - Bonus Track,"[Kendrick Lamar, Dr. Dre]",1,352853,True
65,Kendrick Lamar,"good kid, m.A.A.d city (Deluxe)",spotify:track:6P6HGbOoWUynE2U7yLwLnY,Black Boy Fly - Bonus Track,[Kendrick Lamar],2,279146,True
66,Kendrick Lamar,"good kid, m.A.A.d city (Deluxe)",spotify:track:0NfYAsKygCYwPA2BgTZ1qg,Now Or Never - Bonus Track,"[Kendrick Lamar, Mary J. Blige]",3,256213,False
67,Kendrick Lamar,"good kid, m.A.A.d city (Deluxe)",spotify:track:4wshBvO4qMrxJnPlfkeUWi,"Bitch, Don’t Kill My Vibe - Remix","[Kendrick Lamar, JAY-Z]",4,278586,True


In [0]:
df_kendrick['album'].value_counts()

good kid, m.A.A.d city (Deluxe)                       17
To Pimp A Butterfly                                   16
Black Panther The Album Music From And Inspired By    14
DAMN. COLLECTORS EDITION.                             14
untitled unmastered.                                   8
Name: album, dtype: int64

**Below here will be repeitions of above code for different hip hop artists.**

In [0]:
# Now I want to speed up doing this for multiple artists. For a diverse dataset
# I will get hip hop artists and and then another df with popartists.
# Unfortunately some songs have many artists, and the searched for artist does
# not always appear first in the artist search query, so I have to visually 
# inspect the result, instead of being able to loop through many names and 
# grabbing the first name for every result.
name = "J. Cole"

result = sp.search(name)
result['tracks']['items'][0]['artists'][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6l3HvQ5sa6mXTsMTB19rO5'},
 'href': 'https://api.spotify.com/v1/artists/6l3HvQ5sa6mXTsMTB19rO5',
 'id': '6l3HvQ5sa6mXTsMTB19rO5',
 'name': 'J. Cole',
 'type': 'artist',
 'uri': 'spotify:artist:6l3HvQ5sa6mXTsMTB19rO5'}

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]['uri']
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])

In [0]:
album_names

["Revenge Of The Dreamers III: Director's Cut",
 "Revenge Of The Dreamers III: Director's Cut",
 'Revenge Of The Dreamers III',
 'Revenge Of The Dreamers III',
 'KOD',
 'KOD',
 '4 Your Eyez Only',
 '4 Your Eyez Only',
 'Forest Hills Drive: Live from Fayetteville, NC',
 '2014 Forest Hills Drive',
 '2014 Forest Hills Drive',
 'The Blow Up',
 'Born Sinner',
 'Born Sinner',
 'Born Sinner',
 'Cole World: The Sideline Story',
 'Cole World: The Sideline Story',
 'The Come Up']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

30
30
18
18
12
12
10
10
13
13
13
11
16
21
16
16
16
18


**Unique albums that I want are at 0,4,6,9,11,12,15,17.**

In [0]:
indices = [0,4,6,9,11,12,15,17]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

["Revenge Of The Dreamers III: Director's Cut",
 'KOD',
 '4 Your Eyez Only',
 '2014 Forest Hills Drive',
 'The Blow Up',
 'Born Sinner',
 'Cole World: The Sideline Story',
 'The Come Up']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 

Album Revenge Of The Dreamers III: Director's Cut songs has been added to spotify_tracks
Album KOD songs has been added to spotify_tracks
Album 4 Your Eyez Only songs has been added to spotify_tracks
Album 2014 Forest Hills Drive songs has been added to spotify_tracks
Album The Blow Up songs has been added to spotify_tracks
Album Born Sinner songs has been added to spotify_tracks
Album Cole World: The Sideline Story songs has been added to spotify_tracks
Album The Come Up songs has been added to spotify_tracks


In [0]:
df_jcole = json_normalize(spotify_tracks)
df_jcole

,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,J. Cole,Revenge Of The Dreamers III: Director's Cut,spotify:track:5pu7aEkjSMFVxFiboIDixE,Under The Sun (with J. Cole & Lute feat. DaBaby),"[Dreamville, J. Cole, Lute, DaBaby]",1,201773,True
1,J. Cole,Revenge Of The Dreamers III: Director's Cut,spotify:track:1SYZeqExX1250udNlaEAze,"Down Bad (feat. JID, Bas, J. Cole, EARTHGANG &...","[Dreamville, JID, Bas, J. Cole, EARTHGANG, You...",2,169146,True
2,J. Cole,Revenge Of The Dreamers III: Director's Cut,spotify:track:7KNE7h5dcOP8HgfrlAkHuE,LamboTruck (with Cozz feat. REASON & Childish ...,"[Dreamville, Cozz, REASON, Childish Major]",3,229386,True
3,J. Cole,Revenge Of The Dreamers III: Director's Cut,spotify:track:4ucPhGibPjInOykvVNViRU,"Swivel (with EARTHGANG) - From The Album ""Mirr...","[Dreamville, EARTHGANG]",4,194613,True
4,J. Cole,Revenge Of The Dreamers III: Director's Cut,spotify:track:5bnMRsrpblLaINuPh6b6QA,Oh Wow...Swerve (with J. Cole feat. Zoink Gang...,"[Dreamville, J. Cole, Zoink Gang, KEY!, Maxo K...",5,275986,True
...,...,...,...,...,...,...,...,...
121,J. Cole,The Come Up,spotify:track:3Tfo3wrVM1hPJobHWiLMKS,Carolina On My Mind,[J. Cole],14,221320,False
122,J. Cole,The Come Up,spotify:track:7jKNXf01cKMs9ChaTVArwQ,Can't Cry,[J. Cole],15,255573,False
123,J. Cole,The Come Up,spotify:track:61Z5RMvhwCzoZfvAzMmATn,Goin Off,[J. Cole],16,169040,False
124,J. Cole,The Come Up,spotify:track:2fO5nU9GmjFEsKtYoSg2zB,Get It,[J. Cole],17,217160,False


In [0]:
name = "Drake"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1RyvyyTE3xzB2ZywiAwp0i'},
  'href': 'https://api.spotify.com/v1/artists/1RyvyyTE3xzB2ZywiAwp0i',
  'id': '1RyvyyTE3xzB2ZywiAwp0i',
  'name': 'Future',
  'type': 'artist',
  'uri': 'spotify:artist:1RyvyyTE3xzB2ZywiAwp0i'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/3TVXtAsR1Inumwj472S9r4'},
  'href': 'https://api.spotify.com/v1/artists/3TVXtAsR1Inumwj472S9r4',
  'id': '3TVXtAsR1Inumwj472S9r4',
  'name': 'Drake',
  'type': 'artist',
  'uri': 'spotify:artist:3TVXtAsR1Inumwj472S9r4'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][1]['uri']
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])

In [0]:
album_names

['Care Package',
 'Care Package',
 'So Far Gone',
 'So Far Gone',
 'Scorpion',
 'Scorpion',
 'More Life',
 'More Life',
 'Views',
 'Views',
 'Views',
 'Views',
 'What A Time To Be Alive',
 'What A Time To Be Alive',
 "If You're Reading This It's Too Late",
 "If You're Reading This It's Too Late",
 'Nothing Was The Same (Deluxe)',
 'Nothing Was The Same (Deluxe)',
 'Nothing Was The Same (Deluxe)',
 'Nothing Was The Same (Deluxe)']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

17
17
18
18
25
25
22
22
20
20
20
20
11
11
17
17
16
15
16
16


In [0]:
indices = [0,2,4,6,8,12,14,16]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Care Package',
 'So Far Gone',
 'Scorpion',
 'More Life',
 'Views',
 'What A Time To Be Alive',
 "If You're Reading This It's Too Late",
 'Nothing Was The Same (Deluxe)']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 

Album Care Package songs has been added to spotify_tracks
Album So Far Gone songs has been added to spotify_tracks
Album Scorpion songs has been added to spotify_tracks
Album More Life songs has been added to spotify_tracks
Album Views songs has been added to spotify_tracks
Album What A Time To Be Alive songs has been added to spotify_tracks
Album If You're Reading This It's Too Late songs has been added to spotify_tracks
Album Nothing Was The Same (Deluxe) songs has been added to spotify_tracks


In [0]:
df_drake = json_normalize(spotify_tracks)
df_drake

,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Drake,Care Package,spotify:track:1qyFlfPREPbRcS2BNszdYI,Dreams Money Can Buy,[Drake],1,253803,True
1,Drake,Care Package,spotify:track:0Tacxc5lDJ8LwiO1pWNSl8,The Motion,[Drake],2,240956,True
2,Drake,Care Package,spotify:track:4n4BflhWjCHIxrI4v7Xt9s,How Bout Now,[Drake],3,235566,True
3,Drake,Care Package,spotify:track:4Wjhj0WjkyECccfHVIgaTq,Trust Issues,[Drake],4,281518,True
4,Drake,Care Package,spotify:track:4czcw3NVLY0of5hTD7OufN,Days in The East,[Drake],5,353697,True
...,...,...,...,...,...,...,...,...
141,Drake,Nothing Was The Same (Deluxe),spotify:track:5LnXWeA9tuWKI2C1H74iiJ,Too Much,"[Drake, Sampha]",12,261866,True
142,Drake,Nothing Was The Same (Deluxe),spotify:track:4RI9eX7jNcdaQOJifn7t6z,Pound Cake / Paris Morton Music 2,"[Drake, JAY-Z]",13,433800,True
143,Drake,Nothing Was The Same (Deluxe),spotify:track:5Ggfa9cpkpfp5D6Rg0Yyw1,Come Thru,[Drake],14,236360,True
144,Drake,Nothing Was The Same (Deluxe),spotify:track:5ngydCLbzCEdtu8SEOXhPU,All Me,"[Drake, 2 Chainz, Big Sean]",15,271573,True


In [0]:
name = "Kanye West"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1cNDP5yjU5vjeR8qMf4grg'},
  'href': 'https://api.spotify.com/v1/artists/1cNDP5yjU5vjeR8qMf4grg',
  'id': '1cNDP5yjU5vjeR8qMf4grg',
  'name': 'YNW Melly',
  'type': 'artist',
  'uri': 'spotify:artist:1cNDP5yjU5vjeR8qMf4grg'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
  'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
  'id': '5K4W6rqBFWDnAN6FQUkS6x',
  'name': 'Kanye West',
  'type': 'artist',
  'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][1]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])

In [0]:
album_names

['JESUS IS KING',
 'KIDS SEE GHOSTS',
 'ye',
 'The Life Of Pablo',
 'The Life Of Pablo',
 'Yeezus',
 'Kanye West Presents Good Music Cruel Summer',
 'Watch The Throne (Deluxe)',
 'Watch The Throne (Deluxe)',
 'Watch The Throne (Deluxe)',
 'Watch The Throne (Deluxe)',
 'Watch The Throne',
 'Watch The Throne',
 'Watch The Throne',
 'Watch The Throne',
 'My Beautiful Dark Twisted Fantasy',
 'My Beautiful Dark Twisted Fantasy',
 '808s & Heartbreak',
 '808s & Heartbreak',
 'Graduation (French Limited Version)']

**0,1,2,3,5,6,7,15,17,19.**

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

11
7
7
20
19
10
12
16
16
16
16
12
12
12
12
13
13
12
12
15


In [0]:
indices = [0,1,2,3,5,6,7,15,17,19]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['JESUS IS KING',
 'KIDS SEE GHOSTS',
 'ye',
 'The Life Of Pablo',
 'Yeezus',
 'Kanye West Presents Good Music Cruel Summer',
 'Watch The Throne (Deluxe)',
 'My Beautiful Dark Twisted Fantasy',
 '808s & Heartbreak',
 'Graduation (French Limited Version)']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 

Album JESUS IS KING songs has been added to spotify_tracks
Album KIDS SEE GHOSTS songs has been added to spotify_tracks
Album ye songs has been added to spotify_tracks
Album The Life Of Pablo songs has been added to spotify_tracks
Album Yeezus songs has been added to spotify_tracks
Album Kanye West Presents Good Music Cruel Summer songs has been added to spotify_tracks
Album Watch The Throne (Deluxe) songs has been added to spotify_tracks
Album My Beautiful Dark Twisted Fantasy songs has been added to spotify_tracks
Album 808s & Heartbreak songs has been added to spotify_tracks
Album Graduation (French Limited Version) songs has been added to spotify_tracks


In [0]:
df_kanye = json_normalize(spotify_tracks)
df_kanye

,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Kanye West,JESUS IS KING,spotify:track:1WnqWQcWcuQbVzgE7ecfCY,Every Hour,"[Kanye West, Sunday Service Choir]",1,112239,False
1,Kanye West,JESUS IS KING,spotify:track:39JRmdKFka1Oe09FoOCPI4,Selah,[Kanye West],2,164619,False
2,Kanye West,JESUS IS KING,spotify:track:2QpGZOhTCHHiKmpSO9FW4h,Follow God,[Kanye West],3,104591,False
3,Kanye West,JESUS IS KING,spotify:track:3JWiDGQX2eTlFvKj3Yssj3,Closed On Sunday,[Kanye West],4,151873,False
4,Kanye West,JESUS IS KING,spotify:track:2SasoXZyv82yYgHiVOvxQn,On God,[Kanye West],5,136350,False
...,...,...,...,...,...,...,...,...
118,Kanye West,Graduation (French Limited Version),spotify:track:50pq2uoicYhN7yOsO3cdel,The Glory,[Kanye West],11,212826,True
119,Kanye West,Graduation (French Limited Version),spotify:track:2DvDmy9aCv9TajH0MkHsXa,Homecoming,[Kanye West],12,203506,True
120,Kanye West,Graduation (French Limited Version),spotify:track:3I1hxWXVYhH6rjd3sn3P4n,Big Brother,[Kanye West],13,287413,True
121,Kanye West,Graduation (French Limited Version),spotify:track:6kaltnzR4QATRbBWHDWCp2,Stronger - A-Trak Remix,[Kanye West],14,276146,False


In [0]:
name = "Khalid"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qqNVTkY8uBg9cP3Jd7DAH'},
  'href': 'https://api.spotify.com/v1/artists/6qqNVTkY8uBg9cP3Jd7DAH',
  'id': '6qqNVTkY8uBg9cP3Jd7DAH',
  'name': 'Billie Eilish',
  'type': 'artist',
  'uri': 'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/6LuN9FCkKOj5PcnpouEgny'},
  'href': 'https://api.spotify.com/v1/artists/6LuN9FCkKOj5PcnpouEgny',
  'id': '6LuN9FCkKOj5PcnpouEgny',
  'name': 'Khalid',
  'type': 'artist',
  'uri': 'spotify:artist:6LuN9FCkKOj5PcnpouEgny'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][1]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['Free Spirit',
 'Suncity',
 'Suncity',
 'American Teen (Japan Version)',
 'American Teen']

In [0]:
indices = [0,1,4]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Free Spirit', 'Suncity', 'American Teen']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_khalid = json_normalize(spotify_tracks)
df_khalid

Album Free Spirit songs has been added to spotify_tracks
Album Suncity songs has been added to spotify_tracks
Album American Teen songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Khalid,Free Spirit,spotify:track:5BfMJ6iytQnu1r7iiCYSXe,Intro,[Khalid],1,213253,False
1,Khalid,Free Spirit,spotify:track:2vsuel3g4zaLYSOY0mOMbi,Bad Luck,[Khalid],2,231653,False
2,Khalid,Free Spirit,spotify:track:2XKFnwB6djxrJCjR3PdeRb,My Bad,[Khalid],3,163973,False
3,Khalid,Free Spirit,spotify:track:339Y993TrFyWyO438be0RY,Better,[Khalid],4,229400,False
4,Khalid,Free Spirit,spotify:track:6TqXcAFInzjp0bODyvrWEq,Talk (feat. Disclosure),"[Khalid, Disclosure]",5,197573,False
5,Khalid,Free Spirit,spotify:track:5A1w94uzc1kO1Zhe8WWxC3,Right Back,[Khalid],6,215226,False
6,Khalid,Free Spirit,spotify:track:1E6DEDWDKHoOW0fcFuDghV,Don't Pretend (feat. SAFE),"[Khalid, SAFE]",7,165840,False
7,Khalid,Free Spirit,spotify:track:6fOOKqQWcSy33ApU1wqkkW,Paradise,[Khalid],8,173653,False
8,Khalid,Free Spirit,spotify:track:0HEckRWFSknIupbGzavk5d,Hundred,[Khalid],9,277080,False
9,Khalid,Free Spirit,spotify:track:09oZ9eXQ2fo6YDrPzJqAoP,Outta My Head (with John Mayer),"[Khalid, John Mayer]",10,177066,False


In [0]:
# Concatenating all the data for hip hop artists together
df_hiphop = pd.concat([df_kendrick, df_jcole, df_drake, df_kanye, df_khalid])
df_hiphop = df_hiphop.set_index("uri")
df_hiphop

,artist,album,name,artists,track_number,duration_ms,explicit
uri,,,,,,,
spotify:track:5qN4HFkapdAOV94XPryVof,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,[Kendrick Lamar],1,130613,True
spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars (with SZA),"[Kendrick Lamar, SZA]",2,232186,True
spotify:track:4LmAnpjlhWTahvRkYR8xJa,Kendrick Lamar,Black Panther The Album Music From And Inspire...,X (with 2 Chainz & Saudi),"[ScHoolboy Q, 2 Chainz, Saudi]",3,267426,True
spotify:track:5jyyPsIGM2yqkZN9R3TmvN,Kendrick Lamar,Black Panther The Album Music From And Inspire...,The Ways (with Swae Lee),"[Khalid, Swae Lee]",4,238893,True
spotify:track:7bUcBztfGqO7cSI2gMZeCI,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Opps (with Yugen Blakrok),"[Vince Staples, Yugen Blakrok]",5,180893,True
...,...,...,...,...,...,...,...
spotify:track:4V3VshJLUTCIWa77YiAkvM,Khalid,American Teen,Winter,[Khalid],11,241866,False
spotify:track:25YvGDl2zSE0pH8jrMZ6aY,Khalid,American Teen,Therapy,[Khalid],12,257959,False
spotify:track:6SWHTMywuTXLFlMQ75cAuu,Khalid,American Teen,Keep Me,[Khalid],13,276226,False


In [0]:
df_hiphop["artist"].value_counts()

Drake             146
J. Cole           126
Kanye West        123
Kendrick Lamar     69
Khalid             39
Name: artist, dtype: int64

**Now I want the audio features of all the songs**

In [0]:
# Testing grabbing the audio features for one song
sp.audio_features(df_hiphop.index[0])

[{'acousticness': 0.625,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5qN4HFkapdAOV94XPryVof',
  'danceability': 0.618,
  'duration_ms': 130613,
  'energy': 0.582,
  'id': '5qN4HFkapdAOV94XPryVof',
  'instrumentalness': 3.84e-06,
  'key': 1,
  'liveness': 0.265,
  'loudness': -9.454,
  'mode': 1,
  'speechiness': 0.297,
  'tempo': 90.035,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/5qN4HFkapdAOV94XPryVof',
  'type': 'audio_features',
  'uri': 'spotify:track:5qN4HFkapdAOV94XPryVof',
  'valence': 0.48}]

In [0]:
# Testing grabbing the popularity of a song
pop = sp.track(df_hiphop.index[0])
pop["popularity"]

57

In [0]:
# Function to grab the audio features for all the songs in the dataframe
def audio_features(df):
  tracks = []

  for uri in df.index:
    track_features = {}
    # pull audio features per track
    features = sp.audio_features(uri)
    
    # assign dictionary values for each audio feature
    track_features["uri"] = uri
    track_features['key'] = features[0]['key']
    track_features['mode'] = features[0]['mode']
    track_features['time_signature'] = features[0]['time_signature']
    track_features['acousticness'] = features[0]['acousticness']
    track_features['danceability'] = features[0]['danceability']
    track_features['energy'] = features[0]['energy']
    track_features['instrumentalness'] = features[0]['instrumentalness']
    track_features['liveness'] = features[0]['liveness']
    track_features['loudness'] = features[0]['loudness']
    track_features['speechiness'] = features[0]['speechiness']
    track_features['tempo'] = features[0]['tempo']
    track_features['valence'] = features[0]['valence']
    
    # grabbing popularity
    pop = sp.track(uri)
    track_features['popularity'] = pop['popularity']
    tracks.append(track_features)

  return tracks

In [0]:
tracks = audio_features(df_hiphop)
df_tracks = json_normalize(tracks)

retrying ...5secs
retrying ...5secs
retrying ...3secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [0]:
df_tracks = df_tracks.set_index("uri")
df_tracks

,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
uri,,,,,,,,,,,,,
spotify:track:5qN4HFkapdAOV94XPryVof,1,1,4,0.6250,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57
spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,8,1,4,0.0605,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78
spotify:track:4LmAnpjlhWTahvRkYR8xJa,2,1,4,0.0201,0.768,0.471,0.000000,0.2680,-8.406,0.2590,131.023,0.405,69
spotify:track:5jyyPsIGM2yqkZN9R3TmvN,11,0,4,0.0626,0.727,0.720,0.000001,0.1760,-5.856,0.0488,140.080,0.589,65
spotify:track:7bUcBztfGqO7cSI2gMZeCI,1,1,4,0.1520,0.706,0.775,0.000033,0.4160,-6.819,0.3350,127.929,0.847,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:4V3VshJLUTCIWa77YiAkvM,1,1,4,0.1010,0.678,0.575,0.000009,0.1160,-7.086,0.1230,100.007,0.367,63
spotify:track:25YvGDl2zSE0pH8jrMZ6aY,9,0,4,0.8910,0.707,0.484,0.003440,0.2560,-6.579,0.0481,92.988,0.430,64
spotify:track:6SWHTMywuTXLFlMQ75cAuu,2,0,4,0.3680,0.778,0.574,0.000000,0.1260,-7.775,0.0327,112.048,0.663,62


In [0]:
# Merging tracks dataframe with audio features dataframe, I set uri as the index
# for both. uri is a unique song identifier, so I can merge cleanly on it
df_hiphop = df_hiphop.merge(df_tracks, left_index=True, 
                            right_index=True, how="outer")
# This will be the hiphop part of my data
df_hiphop["genre"] = "hiphop"
df_hiphop

,artist,album,name,artists,track_number,duration_ms,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:5qN4HFkapdAOV94XPryVof,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,[Kendrick Lamar],1,130613,True,1,1,4,0.6250,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop
spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars (with SZA),"[Kendrick Lamar, SZA]",2,232186,True,8,1,4,0.0605,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop
spotify:track:4LmAnpjlhWTahvRkYR8xJa,Kendrick Lamar,Black Panther The Album Music From And Inspire...,X (with 2 Chainz & Saudi),"[ScHoolboy Q, 2 Chainz, Saudi]",3,267426,True,2,1,4,0.0201,0.768,0.471,0.000000,0.2680,-8.406,0.2590,131.023,0.405,69,hiphop
spotify:track:5jyyPsIGM2yqkZN9R3TmvN,Kendrick Lamar,Black Panther The Album Music From And Inspire...,The Ways (with Swae Lee),"[Khalid, Swae Lee]",4,238893,True,11,0,4,0.0626,0.727,0.720,0.000001,0.1760,-5.856,0.0488,140.080,0.589,65,hiphop
spotify:track:7bUcBztfGqO7cSI2gMZeCI,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Opps (with Yugen Blakrok),"[Vince Staples, Yugen Blakrok]",5,180893,True,1,1,4,0.1520,0.706,0.775,0.000033,0.4160,-6.819,0.3350,127.929,0.847,59,hiphop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:4V3VshJLUTCIWa77YiAkvM,Khalid,American Teen,Winter,[Khalid],11,241866,False,1,1,4,0.1010,0.678,0.575,0.000009,0.1160,-7.086,0.1230,100.007,0.367,63,hiphop
spotify:track:25YvGDl2zSE0pH8jrMZ6aY,Khalid,American Teen,Therapy,[Khalid],12,257959,False,9,0,4,0.8910,0.707,0.484,0.003440,0.2560,-6.579,0.0481,92.988,0.430,64,hiphop
spotify:track:6SWHTMywuTXLFlMQ75cAuu,Khalid,American Teen,Keep Me,[Khalid],13,276226,False,2,0,4,0.3680,0.778,0.574,0.000000,0.1260,-7.775,0.0327,112.048,0.663,62,hiphop


In [0]:
df_hiphop["album"].value_counts()

Revenge Of The Dreamers III: Director's Cut           30
Scorpion                                              25
More Life                                             22
Views                                                 20
The Life Of Pablo                                     20
The Come Up                                           18
So Far Gone                                           18
Free Spirit                                           17
good kid, m.A.A.d city (Deluxe)                       17
If You're Reading This It's Too Late                  17
Care Package                                          17
Nothing Was The Same (Deluxe)                         16
To Pimp A Butterfly                                   16
Watch The Throne (Deluxe)                             16
Born Sinner                                           16
Cole World: The Sideline Story                        16
Graduation (French Limited Version)                   15
American Teen                  

# Getting Pop Song Data from Spotify Web API

**I will just be repeating what I did with the hip hop data, with pop artists.**

In [0]:
name = "Ed Sheeran"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V'},
  'href': 'https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V',
  'id': '6eUKZXaKkcviH0Ku9w2n3V',
  'name': 'Ed Sheeran',
  'type': 'artist',
  'uri': 'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/6LuN9FCkKOj5PcnpouEgny'},
  'href': 'https://api.spotify.com/v1/artists/6LuN9FCkKOj5PcnpouEgny',
  'id': '6LuN9FCkKOj5PcnpouEgny',
  'name': 'Khalid',
  'type': 'artist',
  'uri': 'spotify:artist:6LuN9FCkKOj5PcnpouEgny'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['Free Spirit',
 'Suncity',
 'Suncity',
 'American Teen (Japan Version)',
 'American Teen',
 'No.6 Collaborations Project',
 'No.6 Collaborations Project',
 '÷ (Deluxe)',
 'x (Wembley Edition)',
 '5',
 'x (Deluxe Edition)',
 'x (Wembley Edition)',
 'x (Wembley Edition)',
 'x (Wembley Edition)',
 'Live at the Bedford',
 'Loose Change',
 '+',
 '+',
 '+',
 '+',
 '+',
 'No.5 Collaborations Project']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

15
15
16
23
32
16
24
24
24
6
8
18
16
16
12
12
8


**0,2,4,5,10,11,16**

In [0]:
indices = [0,2,4,5,10,11,16]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Free Spirit',
 'Suncity',
 'American Teen',
 'No.6 Collaborations Project',
 'x (Deluxe Edition)',
 'x (Wembley Edition)',
 '+']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_ed = json_normalize(spotify_tracks)
df_ed.head()

Album Free Spirit songs has been added to spotify_tracks
Album Suncity songs has been added to spotify_tracks
Album American Teen songs has been added to spotify_tracks
Album No.6 Collaborations Project songs has been added to spotify_tracks
Album x (Deluxe Edition) songs has been added to spotify_tracks
Album x (Wembley Edition) songs has been added to spotify_tracks
Album + songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Ed Sheeran,Free Spirit,spotify:track:70eFcWOvlMObDhURTqT4Fv,Beautiful People (feat. Khalid),"[Ed Sheeran, Khalid]",1,197866,False
1,Ed Sheeran,Free Spirit,spotify:track:4vUmTMuQqjdnvlZmAH61Qk,South of the Border (feat. Camila Cabello & Ca...,"[Ed Sheeran, Camila Cabello, Cardi B]",2,204466,False
2,Ed Sheeran,Free Spirit,spotify:track:4wuCQX7JvAZLlrcmH4AeZF,Cross Me (feat. Chance the Rapper & PnB Rock),"[Ed Sheeran, Chance the Rapper, PnB Rock]",3,206186,True
3,Ed Sheeran,Free Spirit,spotify:track:1AI7UPw3fgwAFkvAlZWhE0,Take Me Back to London (feat. Stormzy),"[Ed Sheeran, Stormzy]",4,189733,False
4,Ed Sheeran,Free Spirit,spotify:track:0VsGaRXR5WAzpu51unJTis,Best Part of Me (feat. YEBBA),"[Ed Sheeran, Yebba]",5,243266,False


In [0]:
name = "Shawn Mendes"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/7n2wHs1TKAczGzO7Dd2rGr'},
  'href': 'https://api.spotify.com/v1/artists/7n2wHs1TKAczGzO7Dd2rGr',
  'id': '7n2wHs1TKAczGzO7Dd2rGr',
  'name': 'Shawn Mendes',
  'type': 'artist',
  'uri': 'spotify:artist:7n2wHs1TKAczGzO7Dd2rGr'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
  'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
  'id': '4nDoRrQiYLoBzwC5BhVJzF',
  'name': 'Camila Cabello',
  'type': 'artist',
  'uri': 'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['Shawn Mendes (Deluxe)',
 'Shawn Mendes',
 'MTV Unplugged',
 'Illuminate (Deluxe)',
 'Illuminate',
 'Live At Madison Square Garden',
 'Handwritten (Revisited)',
 'Handwritten',
 'Handwritten',
 'Handwritten (Deluxe)',
 'Handwritten (Deluxe)']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

16
14
11
16
13
12
16
13
12
16
15


**0,3,6**

In [0]:
indices = [0,3,6]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Shawn Mendes (Deluxe)', 'Illuminate (Deluxe)', 'Handwritten (Revisited)']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_shawn = json_normalize(spotify_tracks)
df_shawn.head()

Album Shawn Mendes (Deluxe) songs has been added to spotify_tracks
Album Illuminate (Deluxe) songs has been added to spotify_tracks
Album Handwritten (Revisited) songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Shawn Mendes,Shawn Mendes (Deluxe),spotify:track:6v3KW9xbzN5yKLt9YKDYA2,Señorita,"[Shawn Mendes, Camila Cabello]",1,190799,False
1,Shawn Mendes,Shawn Mendes (Deluxe),spotify:track:0TFpGNlWslRQO4oyTtJq6K,Lost In Japan,[Shawn Mendes],2,200840,False
2,Shawn Mendes,Shawn Mendes (Deluxe),spotify:track:2bT1PH7Cw3J9p3t7nlXCdh,If I Can't Have You,[Shawn Mendes],3,191466,False
3,Shawn Mendes,Shawn Mendes (Deluxe),spotify:track:2SFnIqRWyHt9MUNeXkykii,In My Blood,[Shawn Mendes],4,211293,False
4,Shawn Mendes,Shawn Mendes (Deluxe),spotify:track:49u62ZMTXz91U8V4uXaJ4q,Fallin' All In You,[Shawn Mendes],5,235760,False


In [0]:
name = "Camila"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
  'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
  'id': '4nDoRrQiYLoBzwC5BhVJzF',
  'name': 'Camila Cabello',
  'type': 'artist',
  'uri': 'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/4r63FhuTkUYltbVAg5TQnk'},
  'href': 'https://api.spotify.com/v1/artists/4r63FhuTkUYltbVAg5TQnk',
  'id': '4r63FhuTkUYltbVAg5TQnk',
  'name': 'DaBaby',
  'type': 'artist',
  'uri': 'spotify:artist:4r63FhuTkUYltbVAg5TQnk'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['Romance', 'Romance', 'Camila']

In [0]:
indices = [0,2]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]

spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_camila = json_normalize(spotify_tracks)
df_camila.head()

Album Romance songs has been added to spotify_tracks
Album Camila songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Camila,Romance,spotify:track:2ogKhhoMClkFXek7ZgxAhN,Shameless,[Camila Cabello],1,219742,False
1,Camila,Romance,spotify:track:4jFD7byoPdGkEdjHLMduR9,Living Proof,[Camila Cabello],2,194440,False
2,Camila,Romance,spotify:track:0sHc1EEeOnmgjP6QmsR7px,Should've Said It,[Camila Cabello],3,200626,False
3,Camila,Romance,spotify:track:3yOlyBJuViE2YSGn3nVE1K,My Oh My (feat. DaBaby),"[Camila Cabello, DaBaby]",4,170746,False
4,Camila,Romance,spotify:track:3BVgrFWuH01GmCUy9Y2EE8,Señorita,"[Shawn Mendes, Camila Cabello]",5,190813,False


In [0]:
name = "Taylor"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
  'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
  'id': '06HL4z0CvFAxyc27GXpf02',
  'name': 'Taylor Swift',
  'type': 'artist',
  'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['Lover',
 'reputation',
 'reputation Stadium Tour Surprise Song Playlist',
 'reputation (Big Machine Radio Release Special)',
 'Taylor Swift Karaoke: 1989 (Deluxe)',
 '1989',
 '1989',
 'Taylor Swift Karaoke: 1989 (Deluxe)',
 'Taylor Swift Karaoke: 1989 (Deluxe Edition)',
 'Taylor Swift Karaoke: 1989',
 '1989 (Deluxe)',
 '1989 (Deluxe Edition)',
 '1989 (Deluxe Edition)',
 '1989 (Big Machine Radio Release Special)',
 '1989',
 '1989 (Deluxe)',
 'Red (Deluxe Edition)',
 'Red (Deluxe Edition)',
 'Red',
 'Red (Karaoke Version)']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

18
15
46
31
16
13
13
16
16
13
19
19
15
26
13
19
22
22
16
16


**0,1,10,16**

In [0]:
indices = [0,1,10,16]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Lover', 'reputation', '1989 (Deluxe)', 'Red (Deluxe Edition)']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_taylor = json_normalize(spotify_tracks)
df_taylor.head()

Album Lover songs has been added to spotify_tracks
Album reputation songs has been added to spotify_tracks
Album 1989 (Deluxe) songs has been added to spotify_tracks
Album Red (Deluxe Edition) songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Taylor,Lover,spotify:track:43rA71bccXFGD4C8GOpIlN,I Forgot That You Existed,[Taylor Swift],1,170640,False
1,Taylor,Lover,spotify:track:1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,[Taylor Swift],2,178426,False
2,Taylor,Lover,spotify:track:1dGr1c8CrMLDpV6mPbImSI,Lover,[Taylor Swift],3,221306,False
3,Taylor,Lover,spotify:track:3RauEVgRgj1IuWdJ9fDs70,The Man,[Taylor Swift],4,190360,False
4,Taylor,Lover,spotify:track:3pHkh7d0lzM2AldUtz2x37,The Archer,[Taylor Swift],5,211240,False


In [0]:
name = "billie eilish"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/6qqNVTkY8uBg9cP3Jd7DAH'},
  'href': 'https://api.spotify.com/v1/artists/6qqNVTkY8uBg9cP3Jd7DAH',
  'id': '6qqNVTkY8uBg9cP3Jd7DAH',
  'name': 'Billie Eilish',
  'type': 'artist',
  'uri': 'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
# Store artist's albums' names' and uris in separate lists
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?']

In [0]:
spotify_tracks = []
album_count = 0
for album in album_uris:
    spotify_tracks.extend(albumSongs(album, name, album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_billie = json_normalize(spotify_tracks)
df_billie.head()

Album Lover songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,billie eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",spotify:track:0rQtoQXQfwpDW0c7Fw1NeM,!!!!!!!,[Billie Eilish],1,13578,False
1,billie eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",spotify:track:2Fxmhks0bxGSBdJ92vM42m,bad guy,[Billie Eilish],2,194087,False
2,billie eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",spotify:track:4QIo4oxwzzafcBWkKjDpXY,xanny,[Billie Eilish],3,243725,False
3,billie eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",spotify:track:3XF5xLJHOQQRbWya6hBp7d,you should see me in a crown,[Billie Eilish],4,180952,False
4,billie eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",spotify:track:6IRdLKIyS4p7XNiP8r6rsx,all the good girls go to hell,[Billie Eilish],5,168839,False


In [0]:
name = "Justin"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/1uNFoZAHBGtllmzznpCI3s'},
  'href': 'https://api.spotify.com/v1/artists/1uNFoZAHBGtllmzznpCI3s',
  'id': '1uNFoZAHBGtllmzznpCI3s',
  'name': 'Justin Bieber',
  'type': 'artist',
  'uri': 'spotify:artist:1uNFoZAHBGtllmzznpCI3s'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/0VRj0yCOv2FXJNP47XQnx5'},
  'href': 'https://api.spotify.com/v1/artists/0VRj0yCOv2FXJNP47XQnx5',
  'id': '0VRj0yCOv2FXJNP47XQnx5',
  'name': 'Quavo',
  'type': 'artist',
  'uri': 'spotify:artist:0VRj0yCOv2FXJNP47XQnx5'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['Changes',
 'Purpose (Deluxe)',
 'Purpose (Deluxe)',
 'Journals',
 'Journals',
 'Believe Acoustic',
 'Believe Acoustic',
 'Believe',
 'Believe (Deluxe Edition)',
 'Believe (Deluxe Edition)',
 'Never Say Never - The Remixes',
 'Never Say Never - The Remixes',
 'Under The Mistletoe',
 'Under The Mistletoe',
 'Under The Mistletoe (Deluxe Edition)',
 'Under The Mistletoe (Deluxe Edition)',
 'Under The Mistletoe (Deluxe Edition)',
 'My World 2.0',
 'My Worlds (International Version)',
 'My Worlds (Oz Version)']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

17
19
19
16
15
11
11
13
17
16
7
7
11
11
16
15
15
10
18
19


**0,1,3,8,10,14,17**

In [0]:
indices = [0,1,3,8,10,14,17]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Changes',
 'Purpose (Deluxe)',
 'Journals',
 'Believe (Deluxe Edition)',
 'Never Say Never - The Remixes',
 'Under The Mistletoe (Deluxe Edition)',
 'My World 2.0']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_justin = json_normalize(spotify_tracks)
df_justin.head()

Album Changes songs has been added to spotify_tracks
Album Purpose (Deluxe) songs has been added to spotify_tracks
Album Journals songs has been added to spotify_tracks
Album Believe (Deluxe Edition) songs has been added to spotify_tracks
Album Never Say Never - The Remixes songs has been added to spotify_tracks
Album Under The Mistletoe (Deluxe Edition) songs has been added to spotify_tracks
Album My World 2.0 songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Justin,Changes,spotify:track:5Py8zRKGkZvgHniVVtvNCN,All Around Me,[Justin Bieber],1,136906,False
1,Justin,Changes,spotify:track:40qyPyljksBEqlNw5sW37T,Habitual,[Justin Bieber],2,168200,False
2,Justin,Changes,spotify:track:2o9LAypwGH4ctV0i9boo6d,Come Around Me,[Justin Bieber],3,200773,False
3,Justin,Changes,spotify:track:4umIPjkehX1r7uhmGvXiSV,Intentions (feat. Quavo),"[Justin Bieber, Quavo]",4,212866,False
4,Justin,Changes,spotify:track:16wAOAZ2OkqoIDN7TpChjR,Yummy,[Justin Bieber],5,208520,False


In [0]:
name = "Ariana"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR'},
  'href': 'https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR',
  'id': '66CXWjxzNUsdJxJ2JdwvnR',
  'name': 'Ariana Grande',
  'type': 'artist',
  'uri': 'spotify:artist:66CXWjxzNUsdJxJ2JdwvnR'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['k bye for now (swt live)',
 'thank u, next',
 'Sweetener',
 'Sweetener',
 'thank u, next',
 'Christmas & Chill (Japan Version)',
 'Dangerous Woman',
 'Dangerous Woman',
 'Dangerous Woman',
 'Dangerous Woman',
 'Dangerous Woman',
 'Dangerous Woman',
 'Dangerous Woman',
 'My Everything (Italian Edition)',
 'The Remix',
 'My Everything (Deluxe)',
 'My Everything',
 'My Everything',
 'My Everything (Deluxe)',
 'My Everything']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

32
12
15
15
12
7
16
15
11
15
15
11
15
18
15
18
18
15
15
12


**1,2,5,14,15**

In [0]:
indices = [1,2,5,14,15]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['thank u, next',
 'Sweetener',
 'Christmas & Chill (Japan Version)',
 'The Remix',
 'My Everything (Deluxe)']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_ariana = json_normalize(spotify_tracks)
df_ariana.head()

Album thank u, next songs has been added to spotify_tracks
Album Sweetener songs has been added to spotify_tracks
Album Christmas & Chill (Japan Version) songs has been added to spotify_tracks
Album The Remix songs has been added to spotify_tracks
Album My Everything (Deluxe) songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Ariana,"thank u, next",spotify:track:39LmTF9RgyakzSYX8txrow,imagine,[Ariana Grande],1,212266,True
1,Ariana,"thank u, next",spotify:track:1TEL6MlSSVLSdhOSddidlJ,needy,[Ariana Grande],2,171573,True
2,Ariana,"thank u, next",spotify:track:4uTvPEr01pjTbZgl7jcKBD,NASA,[Ariana Grande],3,182000,False
3,Ariana,"thank u, next",spotify:track:2hloaUoRonYssMuqLCBLTX,bloodline,[Ariana Grande],4,216893,True
4,Ariana,"thank u, next",spotify:track:3wFLWP0FcIqHK1wb1CPthQ,fake smile,[Ariana Grande],5,208880,True


In [0]:
name = "ZAYN"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/3KwmxIhSe9UTSEF37kwngR'},
  'href': 'https://api.spotify.com/v1/artists/3KwmxIhSe9UTSEF37kwngR',
  'id': '3KwmxIhSe9UTSEF37kwngR',
  'name': 'SHAED',
  'type': 'artist',
  'uri': 'spotify:artist:3KwmxIhSe9UTSEF37kwngR'},
 {'external_urls': {'spotify': 'https://open.spotify.com/artist/5ZsFI1h6hIdQRw2ti0hz81'},
  'href': 'https://api.spotify.com/v1/artists/5ZsFI1h6hIdQRw2ti0hz81',
  'id': '5ZsFI1h6hIdQRw2ti0hz81',
  'name': 'ZAYN',
  'type': 'artist',
  'uri': 'spotify:artist:5ZsFI1h6hIdQRw2ti0hz81'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][1]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['Icarus Falls',
 'Icarus Falls',
 'Icarus Falls',
 'LIKE I WOULD (The Remixes)',
 'Mind Of Mine (Japan Version)',
 'Mind Of Mine (Deluxe Edition)',
 'Mind Of Mine (Deluxe Edition)']

In [0]:
indices = [0,3,5]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Icarus Falls', 'LIKE I WOULD (The Remixes)', 'Mind Of Mine (Deluxe Edition)']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_zayn = json_normalize(spotify_tracks)
df_zayn.head()

Album Icarus Falls songs has been added to spotify_tracks
Album LIKE I WOULD (The Remixes) songs has been added to spotify_tracks
Album Mind Of Mine (Deluxe Edition) songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,ZAYN,Icarus Falls,spotify:track:5jOoj7wU7fMKngSlo4P9RL,Let Me,[ZAYN],1,185373,False
1,ZAYN,Icarus Falls,spotify:track:2JmltWWwKlEc8GQljSr9Gd,Natural,[ZAYN],2,193186,False
2,ZAYN,Icarus Falls,spotify:track:7EzwENgVhyziTP7tdMAzYF,Back To Life,[ZAYN],3,195466,False
3,ZAYN,Icarus Falls,spotify:track:6eAFrt3Rxv8tXgl9AWpIj5,Common,[ZAYN],4,232520,False
4,ZAYN,Icarus Falls,spotify:track:42BVO7pxzd5YF36ugljnZW,Imprint,[ZAYN],5,190120,False


In [0]:
name = "Halsey"
result = sp.search(name)
result['tracks']['items'][0]['artists']

[{'external_urls': {'spotify': 'https://open.spotify.com/artist/26VFTg2z8YR0cCuwLzESi2'},
  'href': 'https://api.spotify.com/v1/artists/26VFTg2z8YR0cCuwLzESi2',
  'id': '26VFTg2z8YR0cCuwLzESi2',
  'name': 'Halsey',
  'type': 'artist',
  'uri': 'spotify:artist:26VFTg2z8YR0cCuwLzESi2'}]

In [0]:
artist_uri = result['tracks']['items'][0]['artists'][0]["uri"]
sp_albums = sp.artist_albums(artist_uri, album_type='album')
album_names = []
album_uris = []
for i in range(len(sp_albums['items'])):
    album_names.append(sp_albums['items'][i]['name'])
    album_uris.append(sp_albums['items'][i]['uri'])
album_names

['Manic',
 'hopeless fountain kingdom (Deluxe)',
 'hopeless fountain kingdom',
 'hopeless fountain kingdom',
 'hopeless fountain kingdom',
 'hopeless fountain kingdom (Deluxe)',
 'hopeless fountain kingdom',
 'BADLANDS (Deluxe)',
 'BADLANDS (Deluxe)',
 'BADLANDS',
 'BADLANDS']

In [0]:
for album in album_uris:
  print (len(sp.album_tracks(album)["items"]))

16
16
16
13
13
16
13
16
16
11
11


In [0]:
indices = [0,2,7]
new_album_uris = [album_uris[x] for x in indices]
new_album_names = [album_names[x] for x in indices]
new_album_names

['Manic', 'hopeless fountain kingdom', 'BADLANDS (Deluxe)']

In [0]:
spotify_tracks = []
album_count = 0
for album in new_album_uris:
    spotify_tracks.extend(albumSongs(album, name, new_album_names[album_count]))
    print("Album " + str(new_album_names[album_count]) + " songs has been added to spotify_tracks")
    album_count += 1 
df_halsey = json_normalize(spotify_tracks)
df_halsey.head()

Album Manic songs has been added to spotify_tracks
Album hopeless fountain kingdom songs has been added to spotify_tracks
Album BADLANDS (Deluxe) songs has been added to spotify_tracks


,artist,album,uri,name,artists,track_number,duration_ms,explicit
0,Halsey,Manic,spotify:track:55ZcA6PazdlXjE1jO5GrzJ,Ashley,[Halsey],1,186187,True
1,Halsey,Manic,spotify:track:66c7n8ZXmYQQeXyNVfiArc,clementine,[Halsey],2,234410,False
2,Halsey,Manic,spotify:track:2ufmtcIFdFpuUYBPXK5f67,Graveyard,[Halsey],3,181805,False
3,Halsey,Manic,spotify:track:7lidXGPXPYLNThITAOTlkK,You should be sad,[Halsey],4,205473,True
4,Halsey,Manic,spotify:track:6BORJ0ZazxMBpM3cWg79Id,Forever ... (is a long time),[Halsey],5,167904,False


In [0]:
df_pop = pd.concat([df_ed, df_ariana, df_justin, df_billie, df_taylor, 
                       df_camila, df_shawn, df_zayn, df_halsey])
df_pop = df_pop.set_index("uri")
df_pop

,artist,album,name,artists,track_number,duration_ms,explicit
uri,,,,,,,
spotify:track:70eFcWOvlMObDhURTqT4Fv,Ed Sheeran,Free Spirit,Beautiful People (feat. Khalid),"[Ed Sheeran, Khalid]",1,197866,False
spotify:track:4vUmTMuQqjdnvlZmAH61Qk,Ed Sheeran,Free Spirit,South of the Border (feat. Camila Cabello & Ca...,"[Ed Sheeran, Camila Cabello, Cardi B]",2,204466,False
spotify:track:4wuCQX7JvAZLlrcmH4AeZF,Ed Sheeran,Free Spirit,Cross Me (feat. Chance the Rapper & PnB Rock),"[Ed Sheeran, Chance the Rapper, PnB Rock]",3,206186,True
spotify:track:1AI7UPw3fgwAFkvAlZWhE0,Ed Sheeran,Free Spirit,Take Me Back to London (feat. Stormzy),"[Ed Sheeran, Stormzy]",4,189733,False
spotify:track:0VsGaRXR5WAzpu51unJTis,Ed Sheeran,Free Spirit,Best Part of Me (feat. YEBBA),"[Ed Sheeran, Yebba]",5,243266,False
...,...,...,...,...,...,...,...
spotify:track:1r1WtSrPbAOYHE7zwq4eqK,Halsey,BADLANDS (Deluxe),Haunting,[Halsey],12,260797,False
spotify:track:2IO7yf562c1zLzpanal1DT,Halsey,BADLANDS (Deluxe),Gasoline,[Halsey],13,199593,True
spotify:track:3SKzTIl9mW0DadTys9SFo3,Halsey,BADLANDS (Deluxe),Control,[Halsey],14,214761,False


In [0]:
# Using same audio_features function used on hiphop dataset
tracks = audio_features(df_pop)
df_tracks = json_normalize(tracks)

retrying ...1secs
retrying ...5secs
retrying ...2secs
retrying ...4secs
retrying ...3secs
retrying ...5secs
retrying ...4secs


In [0]:
df_tracks.head()

,uri,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity
0,spotify:track:70eFcWOvlMObDhURTqT4Fv,5,0,4,0.124,0.640,0.648,0.000000,0.0802,-8.113,0.1870,92.977,0.548,80
1,spotify:track:4vUmTMuQqjdnvlZmAH61Qk,9,0,4,0.148,0.857,0.621,0.000000,0.0865,-6.376,0.0824,97.986,0.668,86
2,spotify:track:4wuCQX7JvAZLlrcmH4AeZF,4,1,4,0.214,0.746,0.787,0.000000,0.0669,-6.373,0.1200,95.005,0.607,69
3,spotify:track:1AI7UPw3fgwAFkvAlZWhE0,8,0,4,0.219,0.885,0.762,0.000000,0.1620,-5.513,0.2160,138.059,0.605,74
4,spotify:track:0VsGaRXR5WAzpu51unJTis,2,1,4,0.762,0.565,0.242,0.000009,0.2850,-8.367,0.0318,81.657,0.237,71


In [0]:
df_tracks = df_tracks.set_index("uri")
df_pop = df_pop.merge(df_tracks, left_index=True, right_index=True, how="outer")
df_pop["genre"] = "pop"
df_pop.head()

,artist,album,name,artists,track_number,duration_ms,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:70eFcWOvlMObDhURTqT4Fv,Ed Sheeran,Free Spirit,Beautiful People (feat. Khalid),"[Ed Sheeran, Khalid]",1,197866,False,5,0,4,0.124,0.640,0.648,0.000000,0.0802,-8.113,0.1870,92.977,0.548,80,pop
spotify:track:4vUmTMuQqjdnvlZmAH61Qk,Ed Sheeran,Free Spirit,South of the Border (feat. Camila Cabello & Ca...,"[Ed Sheeran, Camila Cabello, Cardi B]",2,204466,False,9,0,4,0.148,0.857,0.621,0.000000,0.0865,-6.376,0.0824,97.986,0.668,86,pop
spotify:track:4wuCQX7JvAZLlrcmH4AeZF,Ed Sheeran,Free Spirit,Cross Me (feat. Chance the Rapper & PnB Rock),"[Ed Sheeran, Chance the Rapper, PnB Rock]",3,206186,True,4,1,4,0.214,0.746,0.787,0.000000,0.0669,-6.373,0.1200,95.005,0.607,69,pop
spotify:track:1AI7UPw3fgwAFkvAlZWhE0,Ed Sheeran,Free Spirit,Take Me Back to London (feat. Stormzy),"[Ed Sheeran, Stormzy]",4,189733,False,8,0,4,0.219,0.885,0.762,0.000000,0.1620,-5.513,0.2160,138.059,0.605,74,pop
spotify:track:0VsGaRXR5WAzpu51unJTis,Ed Sheeran,Free Spirit,Best Part of Me (feat. YEBBA),"[Ed Sheeran, Yebba]",5,243266,False,2,1,4,0.762,0.565,0.242,0.000009,0.2850,-8.367,0.0318,81.657,0.237,71,pop


**The song names currently include features, which is not useful since that data is already included in artists. Also, I need a clean song name to scrape the lyrics of the songs.**

In [0]:
# Make new column with just clean song names using regex
import re
df_pop["name"] = df_pop["name"].str.replace(r' \([^)]*\)', "")
df_hiphop["name"] = df_hiphop["name"].str.replace(r' \([^)]*\)', "")
df_pop.head()

,artist,album,name,artists,track_number,duration_ms,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:70eFcWOvlMObDhURTqT4Fv,Ed Sheeran,Free Spirit,Beautiful People,"[Ed Sheeran, Khalid]",1,197866,False,5,0,4,0.124,0.640,0.648,0.000000,0.0802,-8.113,0.1870,92.977,0.548,80,pop
spotify:track:4vUmTMuQqjdnvlZmAH61Qk,Ed Sheeran,Free Spirit,South of the Border,"[Ed Sheeran, Camila Cabello, Cardi B]",2,204466,False,9,0,4,0.148,0.857,0.621,0.000000,0.0865,-6.376,0.0824,97.986,0.668,86,pop
spotify:track:4wuCQX7JvAZLlrcmH4AeZF,Ed Sheeran,Free Spirit,Cross Me,"[Ed Sheeran, Chance the Rapper, PnB Rock]",3,206186,True,4,1,4,0.214,0.746,0.787,0.000000,0.0669,-6.373,0.1200,95.005,0.607,69,pop
spotify:track:1AI7UPw3fgwAFkvAlZWhE0,Ed Sheeran,Free Spirit,Take Me Back to London,"[Ed Sheeran, Stormzy]",4,189733,False,8,0,4,0.219,0.885,0.762,0.000000,0.1620,-5.513,0.2160,138.059,0.605,74,pop
spotify:track:0VsGaRXR5WAzpu51unJTis,Ed Sheeran,Free Spirit,Best Part of Me,"[Ed Sheeran, Yebba]",5,243266,False,2,1,4,0.762,0.565,0.242,0.000009,0.2850,-8.367,0.0318,81.657,0.237,71,pop


In [0]:
# Save to csv so I dont have to repeat everything above
df_hiphop.to_csv("/content/drive/My Drive/df_hiphop.csv")
df_pop.to_csv("/content/drive/My Drive/df_pop.csv")

# Scraping Song Lyrics from MetroLyrics

In [0]:
# Concatenating the hiphop and pop dataframes
df_songs = pd.concat([df_hiphop, df_pop])
df_songs

,artist,album,name,artists,track_number,duration_ms,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:5qN4HFkapdAOV94XPryVof,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,[Kendrick Lamar],1,130613,True,1,1,4,0.6250,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop
spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,"[Kendrick Lamar, SZA]",2,232186,True,8,1,4,0.0605,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop
spotify:track:4LmAnpjlhWTahvRkYR8xJa,Kendrick Lamar,Black Panther The Album Music From And Inspire...,X,"[ScHoolboy Q, 2 Chainz, Saudi]",3,267426,True,2,1,4,0.0201,0.768,0.471,0.000000,0.2680,-8.406,0.2590,131.023,0.405,69,hiphop
spotify:track:5jyyPsIGM2yqkZN9R3TmvN,Kendrick Lamar,Black Panther The Album Music From And Inspire...,The Ways,"[Khalid, Swae Lee]",4,238893,True,11,0,4,0.0626,0.727,0.720,0.000001,0.1760,-5.856,0.0488,140.080,0.589,65,hiphop
spotify:track:7bUcBztfGqO7cSI2gMZeCI,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Opps,"[Vince Staples, Yugen Blakrok]",5,180893,True,1,1,4,0.1520,0.706,0.775,0.000033,0.4160,-6.819,0.3350,127.929,0.847,59,hiphop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:1r1WtSrPbAOYHE7zwq4eqK,Halsey,BADLANDS (Deluxe),Haunting,[Halsey],12,260797,False,9,0,4,0.0150,0.461,0.696,0.000491,0.2100,-5.752,0.0486,180.006,0.311,59,pop
spotify:track:2IO7yf562c1zLzpanal1DT,Halsey,BADLANDS (Deluxe),Gasoline,[Halsey],13,199593,True,10,0,4,0.2230,0.731,0.580,0.000000,0.1290,-7.328,0.0399,119.997,0.319,72,pop
spotify:track:3SKzTIl9mW0DadTys9SFo3,Halsey,BADLANDS (Deluxe),Control,[Halsey],14,214761,False,4,0,4,0.3990,0.549,0.675,0.000000,0.1810,-5.637,0.0332,119.898,0.206,69,pop


In [0]:
df_songs["artist"].value_counts()

Drake             146
J. Cole           126
Kanye West        123
Ed Sheeran        113
Justin            102
Taylor             74
Kendrick Lamar     69
Ariana             67
ZAYN               54
Halsey             48
Shawn Mendes       48
Khalid             39
Camila             25
billie eilish      14
Name: artist, dtype: int64

**I need the complete artist name to web scrape the song lyrics.**

In [0]:
# Mapping incomplete names to their complete names
dictionary = {
    "Kanye": "Kanye West",
    "Justin": "Justin Bieber",
    "Taylor": "Taylor Swift",
    "Ariana": "Ariana Grande",
    "Camila": "Camila Cabello"
}
df_songs["artist"] = df_songs["artist"].apply(
    lambda x: dictionary[x] if x in dictionary else x)

df_songs["artist"].value_counts()

Drake             146
J. Cole           126
Kanye West        123
Ed Sheeran        113
Justin Bieber     102
Taylor Swift       74
Kendrick Lamar     69
Ariana Grande      67
ZAYN               54
Halsey             48
Shawn Mendes       48
Khalid             39
Camila Cabello     25
billie eilish      14
Name: artist, dtype: int64

In [0]:
# Testing web scraping off of metro lyrics
import requests
from bs4 import BeautifulSoup
import re
import string

url = "https://www.metrolyrics.com/i-dont-care-lyrics-ed-sheeran.html"

content = requests.get(url)
soup = BeautifulSoup(content.content, 'html.parser')

# Metrolyrics has nice verse tag, but each song has multiple verses
a = soup.findAll("p", class_="verse")
 
# Formatting the lyrics text, will explain in the loop
c = " ".join([b.text for b in a])
d = re.sub(r'\[.*?\]', "", c).strip().replace("\n", " \n ").lower()
e = d.translate(str.maketrans('', '', string.punctuation))
f = re.sub('[()]', '', e)
g = re.sub(r' +', " ", f)
h = re.sub(r' +', " ", g.replace("\n", " "))
h

'im at a party i dont wanna be at and i dont ever wear a suit and tie yeah wonderin if i could sneak out the back nobodys even lookin me in my eyes can you take my hand finish my drink say shall we dance hell yeah you know i love ya did i ever tell ya you make it better like that dont think i fit in at this party everyones got so much to say yeah i always feel like im nobody mmm who wants to fit in anyway cause i dont care when im with my baby yeah all the bad things disappear and youre making me feel like maybe i am somebody i can deal with the bad nights when im with my baby yeah ooh ooh ooh ooh ooh ooh cause i dont care as long as you just hold me near you can take me anywhere and youre making me feel like im loved by somebody i can deal with the bad nights when im with my baby yeah ooh ooh ooh ooh ooh ooh we at a party we dont wanna be at tryna talk but we cant hear ourselves press your lips id rather kiss em right back with all these people all around im crippled with anxiety but 

In [0]:
# Testing generating a url from the dataframe columns in the format needed
# An example of an url is above. Urls don't have any punctuation except -
songs = zip(df_songs["artist"], df_songs["name"])
url = "https://www.metrolyrics.com/"
a = list(songs)
# Making a list in the right order, including the lyrics string in between
b = a[0][1].split() + ["lyrics"] + a[0][0].split()
# Removing punctuation from each string in list
c = [c.translate(str.maketrans('', '', string.punctuation)) for c in b]
# Joining the list with a - between each string
d = "-".join(c).lower()
d += ".html"
url += d
url

'https://www.metrolyrics.com/black-panther-lyrics-kendrick-lamar.html'

In [0]:
songs = zip(df_songs["artist"], df_songs["name"])

In [0]:
# Web scraping the lyrics for songs off of metrolyrics.com
import time

song_lyrics = []
for song in songs:
  url = "https://www.metrolyrics.com/"
  # Dictionary for the song lyrics
  lyrics_dict = {}

  try:
    a = list(song)
    b = list(a)[1].split() + ["lyrics"] + list(a)[0].split()
    c = [c.translate(str.maketrans('', '', string.punctuation)) for c in b]
    d = "-".join(c).lower()
    d += ".html"
    url += d

    content = requests.get(url)
    soup = BeautifulSoup(content.content, 'html.parser')

    # Genius has nice class lable "lyrics" for div, which makes scraping easy
    a = soup.findAll("p", class_="verse")
    # Song lyrics are text separated by html tags, so I join them with a space
    c = " ".join([b.text for b in a])
    # Removing verse labels, and adding spaces to \n for even spacing
    d = re.sub(r'\[.*?\]', "", c).strip().replace("\n", " \n ").lower()
    # Removing punctuation
    e = d.translate(str.maketrans('', '', string.punctuation))
    # Removing parentheses around any song lyrics
    f = re.sub('[()]', '', e)
    # Replacing multiple spaces with just one, this will be chain_lyrics
    g = re.sub(r' +', " ", f)
    # This will be the clean lyrics without \n
    h = re.sub(r' +', " ", g.replace("\n", " "))
    lyrics_dict["chain_lyrics"] = g
    lyrics_dict["lyrics"] = h

    song_lyrics.append(lyrics_dict)

  # Try except in case there is an error in the scraping, append empty dict
  except Exception as e:
    song_lyrics.append({})
  
  time.sleep(0.5)

In [0]:
df_lyrics = json_normalize(song_lyrics)
df_lyrics[df_lyrics["lyrics"]!=""].count()

chain_lyrics    831
lyrics          831
dtype: int64

**So I managed to get 831 total songs using that url generation, which is good enough. From other analysis I've noted that many songs aren't even on metrolyrics possible due to being not very well known, and other songs were live versions of songs, of which the url generation wouldn't work. There are other reasons why it could've failed. I will stick with the 831 songs.**

In [0]:
df_songs.reset_index().head()

,index,artist,album,name,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre,duration_s
0,0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,1,1,4,0.6250,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop,130.613
1,1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,8,1,4,0.0605,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop,232.186
2,2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,X,1,2,1,4,0.0201,0.768,0.471,0.000000,0.2680,-8.406,0.2590,131.023,0.405,69,hiphop,267.426
3,3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,The Ways,1,11,0,4,0.0626,0.727,0.720,0.000001,0.1760,-5.856,0.0488,140.080,0.589,65,hiphop,238.893
4,4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Opps,1,1,1,4,0.1520,0.706,0.775,0.000033,0.4160,-6.819,0.3350,127.929,0.847,59,hiphop,180.893


In [0]:
df_complete_lyrics = df_songs.reset_index().iloc[:,1:].merge(
    df_lyrics, left_index=True, right_index=True, how="outer")
df_complete_lyrics

,artist,album,name,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre,duration_s,chain_lyrics,lyrics
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,1,1,4,0.6250,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop,130.613,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,8,1,4,0.0605,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop,232.186,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,X,1,2,1,4,0.0201,0.768,0.471,0.000000,0.2680,-8.406,0.2590,131.023,0.405,69,hiphop,267.426,,
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,The Ways,1,11,0,4,0.0626,0.727,0.720,0.000001,0.1760,-5.856,0.0488,140.080,0.589,65,hiphop,238.893,,
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Opps,1,1,1,4,0.1520,0.706,0.775,0.000033,0.4160,-6.819,0.3350,127.929,0.847,59,hiphop,180.893,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,Halsey,BADLANDS (Deluxe),Haunting,0,9,0,4,0.0150,0.461,0.696,0.000491,0.2100,-5.752,0.0486,180.006,0.311,59,pop,260.797,keep on haunting \n keep on haunting me \n kee...,keep on haunting keep on haunting me keep on h...
1044,Halsey,BADLANDS (Deluxe),Gasoline,1,10,0,4,0.2230,0.731,0.580,0.000000,0.1290,-7.328,0.0399,119.997,0.319,72,pop,199.593,are you insane like me been in pain like me \n...,are you insane like me been in pain like me bo...
1045,Halsey,BADLANDS (Deluxe),Control,0,4,0,4,0.3990,0.549,0.675,0.000000,0.1810,-5.637,0.0332,119.898,0.206,69,pop,214.761,they send me away to find them a fortune \n a ...,they send me away to find them a fortune a che...
1046,Halsey,BADLANDS (Deluxe),Young God,1,9,0,4,0.3240,0.449,0.599,0.000004,0.1140,-6.509,0.0405,119.868,0.198,60,pop,180.088,he says oh baby girl you know were gonna be le...,he says oh baby girl you know were gonna be le...


In [0]:
# For this dataset, I'm not interested in songs for which I don't
# have the lyrics, so I drop those rows.
df_complete_lyrics = df_complete_lyrics[df_complete_lyrics["lyrics"]!=""]
df_complete_lyrics

,artist,album,name,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre,duration_s,chain_lyrics,lyrics
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,1,1,4,0.62500,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop,130.613,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,8,1,4,0.06050,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop,232.186,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...
5,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,8,0,4,0.14400,0.601,0.583,0.000097,0.1830,-5.725,0.2630,148.785,0.320,62,hiphop,208.760,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...
12,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,1,1,4,0.00759,0.855,0.478,0.000000,0.1010,-9.116,0.1930,130.018,0.284,67,hiphop,221.826,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...
13,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,2,1,4,0.07620,0.735,0.677,0.000022,0.1110,-4.979,0.0930,100.584,0.188,76,hiphop,211.440,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1043,Halsey,BADLANDS (Deluxe),Haunting,0,9,0,4,0.01500,0.461,0.696,0.000491,0.2100,-5.752,0.0486,180.006,0.311,59,pop,260.797,keep on haunting \n keep on haunting me \n kee...,keep on haunting keep on haunting me keep on h...
1044,Halsey,BADLANDS (Deluxe),Gasoline,1,10,0,4,0.22300,0.731,0.580,0.000000,0.1290,-7.328,0.0399,119.997,0.319,72,pop,199.593,are you insane like me been in pain like me \n...,are you insane like me been in pain like me bo...
1045,Halsey,BADLANDS (Deluxe),Control,0,4,0,4,0.39900,0.549,0.675,0.000000,0.1810,-5.637,0.0332,119.898,0.206,69,pop,214.761,they send me away to find them a fortune \n a ...,they send me away to find them a fortune a che...
1046,Halsey,BADLANDS (Deluxe),Young God,1,9,0,4,0.32400,0.449,0.599,0.000004,0.1140,-6.509,0.0405,119.868,0.198,60,pop,180.088,he says oh baby girl you know were gonna be le...,he says oh baby girl you know were gonna be le...


In [0]:
df_complete_lyrics = df_complete_lyrics.reset_index().iloc[:,1:]
df_complete_lyrics

,artist,album,name,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre,duration_s,chain_lyrics,lyrics
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,1,1,4,0.62500,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop,130.613,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,8,1,4,0.06050,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop,232.186,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,8,0,4,0.14400,0.601,0.583,0.000097,0.1830,-5.725,0.2630,148.785,0.320,62,hiphop,208.760,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,1,1,4,0.00759,0.855,0.478,0.000000,0.1010,-9.116,0.1930,130.018,0.284,67,hiphop,221.826,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,2,1,4,0.07620,0.735,0.677,0.000022,0.1110,-4.979,0.0930,100.584,0.188,76,hiphop,211.440,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,Halsey,BADLANDS (Deluxe),Haunting,0,9,0,4,0.01500,0.461,0.696,0.000491,0.2100,-5.752,0.0486,180.006,0.311,59,pop,260.797,keep on haunting \n keep on haunting me \n kee...,keep on haunting keep on haunting me keep on h...
827,Halsey,BADLANDS (Deluxe),Gasoline,1,10,0,4,0.22300,0.731,0.580,0.000000,0.1290,-7.328,0.0399,119.997,0.319,72,pop,199.593,are you insane like me been in pain like me \n...,are you insane like me been in pain like me bo...
828,Halsey,BADLANDS (Deluxe),Control,0,4,0,4,0.39900,0.549,0.675,0.000000,0.1810,-5.637,0.0332,119.898,0.206,69,pop,214.761,they send me away to find them a fortune \n a ...,they send me away to find them a fortune a che...
829,Halsey,BADLANDS (Deluxe),Young God,1,9,0,4,0.32400,0.449,0.599,0.000004,0.1140,-6.509,0.0405,119.868,0.198,60,pop,180.088,he says oh baby girl you know were gonna be le...,he says oh baby girl you know were gonna be le...


**This dataframe is for textual features, so I can drop the audio features.**

In [0]:
df_complete_lyrics = df_complete_lyrics.iloc[:,[0,1,2,3,17,19,20]]
df_complete_lyrics

,artist,album,name,explicit,genre,chain_lyrics,lyrics
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,hiphop,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,hiphop,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,hiphop,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,hiphop,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,hiphop,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...
...,...,...,...,...,...,...,...
826,Halsey,BADLANDS (Deluxe),Haunting,0,pop,keep on haunting \n keep on haunting me \n kee...,keep on haunting keep on haunting me keep on h...
827,Halsey,BADLANDS (Deluxe),Gasoline,1,pop,are you insane like me been in pain like me \n...,are you insane like me been in pain like me bo...
828,Halsey,BADLANDS (Deluxe),Control,0,pop,they send me away to find them a fortune \n a ...,they send me away to find them a fortune a che...
829,Halsey,BADLANDS (Deluxe),Young God,1,pop,he says oh baby girl you know were gonna be le...,he says oh baby girl you know were gonna be le...


In [0]:
df_complete_lyrics.to_csv("/content/drive/My Drive/df_complete_lyrics.csv")

# Getting Sentiment Analysis from Google Cloud Natural Language API.

In [0]:
APIKEY = 

**The json response object is very simple, with just a polarity and a magnitude. Don't need to practice or inspect the response.**

In [0]:
# Grabbing sentiment analysis or lyrics from Google Cloud Natural Language API.
# I think Google likely will perform better than perhaps a python library
from googleapiclient.discovery import build
lservice = build('language', 'v1beta1', developerKey=APIKEY)

sentiments = []
for lyric in df_complete_lyrics["lyrics"]:
  sentiment= {}
  response = lservice.documents().analyzeSentiment(
    body={
      'document': {
         'type': 'PLAIN_TEXT',
         'content': lyric
      }
    }).execute()
  sentiment["polarity"] = response['documentSentiment']['polarity']
  sentiment["magnitude"] = response['documentSentiment']['magnitude']
  sentiments.append(sentiment)

In [0]:
df_sentiment = json_normalize(sentiments)
df_sentiment

,polarity,magnitude
0,-1.0,1.2
1,-1.0,1.3
2,-1.0,2.2
3,-1.0,1.3
4,-1.0,1.1
...,...,...
826,-1.0,1.4
827,-1.0,1.0
828,-1.0,1.7
829,0.2,1.1


In [0]:
# Values are passed in order of original dataframe, so I can reset index of 
# original and merge the two together. Resetting index creates extra column
# that I drop
df_songs_sentiment = df_complete_lyrics.reset_index().iloc[:,1:].merge(
    df_sentiment, left_index=True, right_index=True, how="outer")
df_songs_sentiment

,artist,album,name,explicit,genre,chain_lyrics,lyrics,polarity,magnitude
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,hiphop,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...,-1.0,1.2
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,hiphop,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...,-1.0,1.3
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,hiphop,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...,-1.0,2.2
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,hiphop,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...,-1.0,1.3
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,hiphop,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...,-1.0,1.1
...,...,...,...,...,...,...,...,...,...
826,Halsey,BADLANDS (Deluxe),Haunting,0,pop,keep on haunting \n keep on haunting me \n kee...,keep on haunting keep on haunting me keep on h...,-1.0,1.4
827,Halsey,BADLANDS (Deluxe),Gasoline,1,pop,are you insane like me been in pain like me \n...,are you insane like me been in pain like me bo...,-1.0,1.0
828,Halsey,BADLANDS (Deluxe),Control,0,pop,they send me away to find them a fortune \n a ...,they send me away to find them a fortune a che...,-1.0,1.7
829,Halsey,BADLANDS (Deluxe),Young God,1,pop,he says oh baby girl you know were gonna be le...,he says oh baby girl you know were gonna be le...,0.2,1.1


# Getting Personality Analysis from IBM Watson API Personality Insights

In [0]:
pip install --upgrade "ibm-watson>=4.3.0"

     |████████████████████████████████| 368kB 4.9MB/s 
     |████████████████████████████████| 204kB 50.0MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-4.3.0-cp36-none-any.whl size=354170 sha256=2a1b2cb668535494ede5fb30228c6634d739f23c324b3ecaeb23818c8b4bfc93
  Stored in directory: /root/.cache/pip/wheels/b7/10/b4/509307276e9d558a6b06d8e69f8faf6cfbdf2d1b83abb6b8d4
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-cp36-none-any.whl size=44491 sha256=ed34176e32474f48e15e23fb2f66e1272e8bbc9c12e668615fb69b8abeb3acea
  Stored in directory: /root/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [0]:
APIKEY = 

In [0]:
# Practicing getting one result for one text "document"
from ibm_watson import PersonalityInsightsV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
# Authenticator object
authenticator = IAMAuthenticator(APIKEY)
personality_insights = PersonalityInsightsV3(
    version= '2017-10-13',
    authenticator=authenticator
)
# Dallas service url, closest to SLO
personality_insights.set_service_url(
    'https://api.us-south.personality-insights.watson.cloud.ibm.com')

profile = personality_insights.profile(
        df_songs_sentiment.loc[0]["lyrics"],
        accept='application/json',
        content_type='text/html',
).get_result()

**The response json contains 5 categories: Agreeableness, Conscientiousness, Extraversion, Emotional range, and Openness. Each category has a list of personality types for which a percentile is calcualted. I will look at the result for one text document and see which personality types I want to extract (whatever ones I think are good predictors.**

In [0]:
# Inspecting the results for each of the big 5 categories
profile["personality"][0]["children"]

[{'category': 'personality',
  'name': 'Adventurousness',
  'percentile': 0.7713670210526818,
  'significant': True,
  'trait_id': 'facet_adventurousness'},
 {'category': 'personality',
  'name': 'Artistic interests',
  'percentile': 0.9999827901171386,
  'significant': True,
  'trait_id': 'facet_artistic_interests'},
 {'category': 'personality',
  'name': 'Emotionality',
  'percentile': 0.9315941301830221,
  'significant': True,
  'trait_id': 'facet_emotionality'},
 {'category': 'personality',
  'name': 'Imagination',
  'percentile': 0.9983841275049014,
  'significant': True,
  'trait_id': 'facet_imagination'},
 {'category': 'personality',
  'name': 'Intellect',
  'percentile': 0.9989852735576126,
  'significant': True,
  'trait_id': 'facet_intellect'},
 {'category': 'personality',
  'name': 'Authority-challenging',
  'percentile': 0.9952214483173324,
  'significant': True,
  'trait_id': 'facet_liberalism'}]

**I want artistic, emotionality, and imagination, which are at
profile["personality"][0]["children"] items 1,2,and 3.**

In [0]:
profile["personality"][1]["children"]

[{'category': 'personality',
  'name': 'Achievement striving',
  'percentile': 0.33591482594587113,
  'significant': True,
  'trait_id': 'facet_achievement_striving'},
 {'category': 'personality',
  'name': 'Cautiousness',
  'percentile': 0.48078344770072584,
  'significant': True,
  'trait_id': 'facet_cautiousness'},
 {'category': 'personality',
  'name': 'Dutifulness',
  'percentile': 0.041552914493275805,
  'significant': True,
  'trait_id': 'facet_dutifulness'},
 {'category': 'personality',
  'name': 'Orderliness',
  'percentile': 0.8674746970454448,
  'significant': True,
  'trait_id': 'facet_orderliness'},
 {'category': 'personality',
  'name': 'Self-discipline',
  'percentile': 0.1329757632808503,
  'significant': True,
  'trait_id': 'facet_self_discipline'},
 {'category': 'personality',
  'name': 'Self-efficacy',
  'percentile': 0.16777654223353877,
  'significant': True,
  'trait_id': 'facet_self_efficacy'}]

**I don't necessarily need anything here.**

In [0]:
profile["personality"][2]["children"]

[{'category': 'personality',
  'name': 'Activity level',
  'percentile': 0.5296401678262467,
  'significant': True,
  'trait_id': 'facet_activity_level'},
 {'category': 'personality',
  'name': 'Assertiveness',
  'percentile': 0.0952716723151425,
  'significant': True,
  'trait_id': 'facet_assertiveness'},
 {'category': 'personality',
  'name': 'Cheerfulness',
  'percentile': 0.3870430243895838,
  'significant': True,
  'trait_id': 'facet_cheerfulness'},
 {'category': 'personality',
  'name': 'Excitement-seeking',
  'percentile': 0.7961856092352693,
  'significant': True,
  'trait_id': 'facet_excitement_seeking'},
 {'category': 'personality',
  'name': 'Outgoing',
  'percentile': 0.10137417440037588,
  'significant': True,
  'trait_id': 'facet_friendliness'},
 {'category': 'personality',
  'name': 'Gregariousness',
  'percentile': 0.05635805754081957,
  'significant': True,
  'trait_id': 'facet_gregariousness'}]

**I want assertiveness, cheerfulness, and outgoing.** At profile["personality"][2]["children"] 1,2,4.**

In [0]:
profile["personality"][3]["children"]

[{'category': 'personality',
  'name': 'Altruism',
  'percentile': 0.538223668884807,
  'significant': True,
  'trait_id': 'facet_altruism'},
 {'category': 'personality',
  'name': 'Cooperation',
  'percentile': 0.23109053962463755,
  'significant': True,
  'trait_id': 'facet_cooperation'},
 {'category': 'personality',
  'name': 'Modesty',
  'percentile': 0.4211426512168481,
  'significant': True,
  'trait_id': 'facet_modesty'},
 {'category': 'personality',
  'name': 'Uncompromising',
  'percentile': 0.5332614966874416,
  'significant': True,
  'trait_id': 'facet_morality'},
 {'category': 'personality',
  'name': 'Sympathy',
  'percentile': 0.9850128762364618,
  'significant': True,
  'trait_id': 'facet_sympathy'},
 {'category': 'personality',
  'name': 'Trust',
  'percentile': 0.33490127396513636,
  'significant': True,
  'trait_id': 'facet_trust'}]

**I want modesty and sympathy.** profile["personality"][3 ]["children"] 2,4

In [0]:
profile["personality"][4]["children"]

[{'category': 'personality',
  'name': 'Fiery',
  'percentile': 0.38635243247105927,
  'significant': True,
  'trait_id': 'facet_anger'},
 {'category': 'personality',
  'name': 'Prone to worry',
  'percentile': 0.42765965594186733,
  'significant': True,
  'trait_id': 'facet_anxiety'},
 {'category': 'personality',
  'name': 'Melancholy',
  'percentile': 0.9215424971788599,
  'significant': True,
  'trait_id': 'facet_depression'},
 {'category': 'personality',
  'name': 'Immoderation',
  'percentile': 0.5428585184312793,
  'significant': True,
  'trait_id': 'facet_immoderation'},
 {'category': 'personality',
  'name': 'Self-consciousness',
  'percentile': 0.8404856660550624,
  'significant': True,
  'trait_id': 'facet_self_consciousness'},
 {'category': 'personality',
  'name': 'Susceptible to stress',
  'percentile': 0.3295975826844284,
  'significant': True,
  'trait_id': 'facet_vulnerability'}]

**I want fiery and melancholy. profile["personality"][4]["children"] 0,2.**

**This gives me a total of 10 personality traits.**

In [0]:
# Testing putting those ten traits into a dictionary
traits = {}
traits["artistic"] = profile["personality"][0]["children"][1]["percentile"]
traits["emotion"] = profile["personality"][0]["children"][2]["percentile"]
traits["imagination"] = profile["personality"][0]["children"][3]["percentile"]

traits["assertive"] = profile["personality"][2]["children"][1]["percentile"]
traits["cheeful"] = profile["personality"][2]["children"][2]["percentile"]
traits["outgoing"] = profile["personality"][2]["children"][4]["percentile"]

traits["modesty"] = profile["personality"][3]["children"][2]["percentile"]
traits["sympathy"] = profile["personality"][3]["children"][4]["percentile"]

traits["fiery"] = profile["personality"][4]["children"][0]["percentile"]
traits["melancholy"] = profile["personality"][4]["children"][2]["percentile"]

traits

{'artistic': 0.9999827901171386,
 'assertive': 0.0952716723151425,
 'cheeful': 0.3870430243895838,
 'emotion': 0.9315941301830221,
 'fiery': 0.38635243247105927,
 'imagination': 0.9983841275049014,
 'melancholy': 0.9215424971788599,
 'modesty': 0.4211426512168481,
 'outgoing': 0.10137417440037588,
 'sympathy': 0.9850128762364618}

**The IBM Watson Personality Insights documentation states that 3000 words are needed for maximum precision, but that above 100 terms is acceptable for an analysis. Less than 100 terms will be an ApiException.**

In [0]:
# Grabbing the 10 chosen personality traits for each lyric
from ibm_watson import ApiException

personality = []

for lyric in df_songs_sentiment["lyrics"]:
  traits = {}

  try:
    profile = personality_insights.profile(
          lyric,
          accept='application/json',
          content_type='text/html',
    ).get_result()  

    traits["artistic"] = profile["personality"][0]["children"][1]["percentile"]
    traits["emotion"] = profile["personality"][0]["children"][2]["percentile"]
    traits["imagination"] = profile["personality"][0]["children"][3]["percentile"]

    traits["assertive"] = profile["personality"][2]["children"][1]["percentile"]
    traits["cheeful"] = profile["personality"][2]["children"][2]["percentile"]
    traits["outgoing"] = profile["personality"][2]["children"][4]["percentile"]

    traits["modesty"] = profile["personality"][3]["children"][2]["percentile"]
    traits["sympathy"] = profile["personality"][3]["children"][4]["percentile"]

    traits["fiery"] = profile["personality"][4]["children"][0]["percentile"]
    traits["melancholy"] = profile["personality"][4]["children"][2]["percentile"]

    personality.append(traits)

  # To continue the program if an ApiException occurrs. Will append an empty 
  # dictionary. The API requires at least 100 tokens to do an analysis, which
  # not all my lyrics might contain.
  except ApiException:
    personality.append({})

ERROR:root:The number of words 75 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 75 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transaction-id: 6324869e339cf62ab63d5bb561412736
ERROR:root:The number of words 74 is less than the minimum number of words required for analysis: 100
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: The number of words 74 is less than the minimum number of words required for analysis: 100, Code: 400 , X-global-transacti

In [0]:
df_personality = json_normalize(personality)
df_personality.isnull().sum()

artistic       10
emotion        10
imagination    10
assertive      10
cheeful        10
outgoing       10
modesty        10
sympathy       10
fiery          10
melancholy     10
dtype: int64

**10 of my lyrics contained less than 100 words, but that is OK.**

In [0]:
df_personality

,artistic,emotion,imagination,assertive,cheeful,outgoing,modesty,sympathy,fiery,melancholy
0,0.999983,0.931594,0.998384,0.095272,0.387043,0.101374,0.421143,0.985013,0.386352,0.921542
1,0.996511,0.986234,0.991554,0.747748,0.986040,0.705842,0.441873,0.842244,0.157679,0.626636
2,0.906971,0.318395,0.994980,0.574279,0.469771,0.320161,0.024560,0.785487,0.582447,0.669157
3,0.967339,0.229457,0.975867,0.382183,0.880943,0.477181,0.000747,0.270318,0.787414,0.197281
4,0.980110,0.889895,0.977369,0.307116,0.962168,0.668731,0.864149,0.875111,0.464351,0.871338
...,...,...,...,...,...,...,...,...,...,...
826,0.930841,0.915972,0.965041,0.019663,0.737102,0.104255,0.964988,0.768947,0.798722,0.982352
827,0.988442,0.748622,0.999587,0.672225,0.720852,0.430028,0.083116,0.693261,0.935605,0.962687
828,0.995897,0.990308,0.999998,0.062743,0.096730,0.006704,0.654713,0.966235,0.990844,0.999999
829,0.974639,0.804268,0.934779,0.226985,0.578715,0.297378,0.678562,0.892613,0.147212,0.884749


In [0]:
df_songs_sentiment

,artist,album,name,explicit,genre,chain_lyrics,lyrics,polarity,magnitude
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,hiphop,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...,-1.0,1.2
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,hiphop,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...,-1.0,1.3
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,hiphop,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...,-1.0,2.2
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,hiphop,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...,-1.0,1.3
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,hiphop,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...,-1.0,1.1
...,...,...,...,...,...,...,...,...,...
826,Halsey,BADLANDS (Deluxe),Haunting,0,pop,keep on haunting \n keep on haunting me \n kee...,keep on haunting keep on haunting me keep on h...,-1.0,1.4
827,Halsey,BADLANDS (Deluxe),Gasoline,1,pop,are you insane like me been in pain like me \n...,are you insane like me been in pain like me bo...,-1.0,1.0
828,Halsey,BADLANDS (Deluxe),Control,0,pop,they send me away to find them a fortune \n a ...,they send me away to find them a fortune a che...,-1.0,1.7
829,Halsey,BADLANDS (Deluxe),Young God,1,pop,he says oh baby girl you know were gonna be le...,he says oh baby girl you know were gonna be le...,0.2,1.1


In [0]:
df_personality_sentiment = df_songs_sentiment.merge(df_personality, 
                          right_index=True, left_index=True, how="outer")
df_personality_sentiment

,artist,album,name,explicit,genre,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,assertive,cheeful,outgoing,modesty,sympathy,fiery,melancholy
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,hiphop,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...,-1.0,1.2,0.999983,0.931594,0.998384,0.095272,0.387043,0.101374,0.421143,0.985013,0.386352,0.921542
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,hiphop,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...,-1.0,1.3,0.996511,0.986234,0.991554,0.747748,0.986040,0.705842,0.441873,0.842244,0.157679,0.626636
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,hiphop,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...,-1.0,2.2,0.906971,0.318395,0.994980,0.574279,0.469771,0.320161,0.024560,0.785487,0.582447,0.669157
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,hiphop,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...,-1.0,1.3,0.967339,0.229457,0.975867,0.382183,0.880943,0.477181,0.000747,0.270318,0.787414,0.197281
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,hiphop,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...,-1.0,1.1,0.980110,0.889895,0.977369,0.307116,0.962168,0.668731,0.864149,0.875111,0.464351,0.871338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,Halsey,BADLANDS (Deluxe),Haunting,0,pop,keep on haunting \n keep on haunting me \n kee...,keep on haunting keep on haunting me keep on h...,-1.0,1.4,0.930841,0.915972,0.965041,0.019663,0.737102,0.104255,0.964988,0.768947,0.798722,0.982352
827,Halsey,BADLANDS (Deluxe),Gasoline,1,pop,are you insane like me been in pain like me \n...,are you insane like me been in pain like me bo...,-1.0,1.0,0.988442,0.748622,0.999587,0.672225,0.720852,0.430028,0.083116,0.693261,0.935605,0.962687
828,Halsey,BADLANDS (Deluxe),Control,0,pop,they send me away to find them a fortune \n a ...,they send me away to find them a fortune a che...,-1.0,1.7,0.995897,0.990308,0.999998,0.062743,0.096730,0.006704,0.654713,0.966235,0.990844,0.999999
829,Halsey,BADLANDS (Deluxe),Young God,1,pop,he says oh baby girl you know were gonna be le...,he says oh baby girl you know were gonna be le...,0.2,1.1,0.974639,0.804268,0.934779,0.226985,0.578715,0.297378,0.678562,0.892613,0.147212,0.884749


In [0]:
# I am not interested in songs that I don't have personality analysis for. Also
# songs shorter than 100 words aren't that great for predictiveness or for
# random song generation.
df_personality_sentiment = df_personality_sentiment \
            [df_personality_sentiment["artistic"].notnull()]
df_personality_sentiment = df_personality_sentiment.reset_index().iloc[:,1:]
df_personality_sentiment

,artist,album,name,explicit,genre,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,assertive,cheeful,outgoing,modesty,sympathy,fiery,melancholy
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,hiphop,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...,-1.0,1.2,0.999983,0.931594,0.998384,0.095272,0.387043,0.101374,0.421143,0.985013,0.386352,0.921542
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,hiphop,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...,-1.0,1.3,0.996511,0.986234,0.991554,0.747748,0.986040,0.705842,0.441873,0.842244,0.157679,0.626636
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,hiphop,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...,-1.0,2.2,0.906971,0.318395,0.994980,0.574279,0.469771,0.320161,0.024560,0.785487,0.582447,0.669157
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,hiphop,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...,-1.0,1.3,0.967339,0.229457,0.975867,0.382183,0.880943,0.477181,0.000747,0.270318,0.787414,0.197281
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,hiphop,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...,-1.0,1.1,0.980110,0.889895,0.977369,0.307116,0.962168,0.668731,0.864149,0.875111,0.464351,0.871338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816,Halsey,BADLANDS (Deluxe),Haunting,0,pop,keep on haunting \n keep on haunting me \n kee...,keep on haunting keep on haunting me keep on h...,-1.0,1.4,0.930841,0.915972,0.965041,0.019663,0.737102,0.104255,0.964988,0.768947,0.798722,0.982352
817,Halsey,BADLANDS (Deluxe),Gasoline,1,pop,are you insane like me been in pain like me \n...,are you insane like me been in pain like me bo...,-1.0,1.0,0.988442,0.748622,0.999587,0.672225,0.720852,0.430028,0.083116,0.693261,0.935605,0.962687
818,Halsey,BADLANDS (Deluxe),Control,0,pop,they send me away to find them a fortune \n a ...,they send me away to find them a fortune a che...,-1.0,1.7,0.995897,0.990308,0.999998,0.062743,0.096730,0.006704,0.654713,0.966235,0.990844,0.999999
819,Halsey,BADLANDS (Deluxe),Young God,1,pop,he says oh baby girl you know were gonna be le...,he says oh baby girl you know were gonna be le...,0.2,1.1,0.974639,0.804268,0.934779,0.226985,0.578715,0.297378,0.678562,0.892613,0.147212,0.884749


# Final Cleaning

In [0]:
df_personality_sentiment.head()

,artist,album,name,explicit,genre,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,assertive,cheeful,outgoing,modesty,sympathy,fiery,melancholy
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,hiphop,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...,-1.0,1.2,0.999983,0.931594,0.998384,0.095272,0.387043,0.101374,0.421143,0.985013,0.386352,0.921542
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,hiphop,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...,-1.0,1.3,0.996511,0.986234,0.991554,0.747748,0.986040,0.705842,0.441873,0.842244,0.157679,0.626636
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,hiphop,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...,-1.0,2.2,0.906971,0.318395,0.994980,0.574279,0.469771,0.320161,0.024560,0.785487,0.582447,0.669157
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,hiphop,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...,-1.0,1.3,0.967339,0.229457,0.975867,0.382183,0.880943,0.477181,0.000747,0.270318,0.787414,0.197281
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,hiphop,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...,-1.0,1.1,0.980110,0.889895,0.977369,0.307116,0.962168,0.668731,0.864149,0.875111,0.464351,0.871338


**The artists and track columns are unnecessary, and I would like to transform duration to seconds instead of milliseconds.**

**Also at this point, I want the boolean explicit column to be 0 and 1.**

In [0]:
df_personality_sentiment["explicit"] = \
            df_personality_sentiment["explicit"].astype("int64")
df_personality_sentiment.head()

,uri,artist,album,name,explicit,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,assertive,cheeful,outgoing,modesty,sympathy,fiery,melancholy,duration_s
0,spotify:track:5qN4HFkapdAOV94XPryVof,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,0.62500,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...,0.0,0.4,0.999983,0.931594,0.998384,0.095272,0.387043,0.101374,0.421143,0.985013,0.386352,0.921542,130.613
1,spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,0.06050,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...,-0.8,0.2,0.996511,0.986234,0.991554,0.747748,0.986040,0.705842,0.441873,0.842244,0.157679,0.626636,232.186
2,spotify:track:0DJBgBiYeSn6n1AXAkFVE8,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,0.14400,0.601,0.583,0.000097,0.1830,-5.725,0.2630,148.785,0.320,62,hiphop,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...,-1.0,1.9,0.906971,0.318395,0.994980,0.574279,0.469771,0.320161,0.024560,0.785487,0.582447,0.669157,208.760
3,spotify:track:5cXg9IQS34FzLVdHhp7hu7,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,0.00759,0.855,0.478,0.000000,0.1010,-9.116,0.1930,130.018,0.284,67,hiphop,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...,0.7,0.4,0.967339,0.229457,0.975867,0.382183,0.880943,0.477181,0.000747,0.270318,0.787414,0.197281,221.826
4,spotify:track:77UjLW8j5UAGAGVGhR5oUK,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,0.07620,0.735,0.677,0.000022,0.1110,-4.979,0.0930,100.584,0.188,76,hiphop,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...,-1.0,0.9,0.980110,0.889895,0.977369,0.307116,0.962168,0.668731,0.864149,0.875111,0.464351,0.871338,211.440


**Above is my final sentiment personality dataset. Now I look at the first dataset without lyrics. I want to do the same things, remove artists and track_number, transform duration to s, and map explicit to 0 and 1.**

In [0]:
df_songs.head()

,artist,album,name,artists,track_number,duration_ms,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre
uri,,,,,,,,,,,,,,,,,,,,,
spotify:track:5qN4HFkapdAOV94XPryVof,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,[Kendrick Lamar],1,130613,True,1,1,4,0.6250,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop
spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,"[Kendrick Lamar, SZA]",2,232186,True,8,1,4,0.0605,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop
spotify:track:4LmAnpjlhWTahvRkYR8xJa,Kendrick Lamar,Black Panther The Album Music From And Inspire...,X,"[ScHoolboy Q, 2 Chainz, Saudi]",3,267426,True,2,1,4,0.0201,0.768,0.471,0.000000,0.2680,-8.406,0.2590,131.023,0.405,69,hiphop
spotify:track:5jyyPsIGM2yqkZN9R3TmvN,Kendrick Lamar,Black Panther The Album Music From And Inspire...,The Ways,"[Khalid, Swae Lee]",4,238893,True,11,0,4,0.0626,0.727,0.720,0.000001,0.1760,-5.856,0.0488,140.080,0.589,65,hiphop
spotify:track:7bUcBztfGqO7cSI2gMZeCI,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Opps,"[Vince Staples, Yugen Blakrok]",5,180893,True,1,1,4,0.1520,0.706,0.775,0.000033,0.4160,-6.819,0.3350,127.929,0.847,59,hiphop


In [0]:
df_songs["explicit"] = df_songs["explicit"].astype("int64")
df_songs["duration_s"] = df_songs["duration_ms"] / 1000
df_songs = df_songs.drop(
    columns = ["artists", "track_number","duration_ms"], axis=1
)
df_songs.head()

,artist,album,name,explicit,key,mode,time_signature,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,genre,duration_s
uri,,,,,,,,,,,,,,,,,,,
spotify:track:5qN4HFkapdAOV94XPryVof,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,1,1,4,0.6250,0.618,0.582,0.000004,0.2650,-9.454,0.2970,90.035,0.480,57,hiphop,130.613
spotify:track:3GCdLUSnKSMJhs4Tj6CV3s,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,8,1,4,0.0605,0.698,0.633,0.000194,0.0926,-4.946,0.0597,96.924,0.552,78,hiphop,232.186
spotify:track:4LmAnpjlhWTahvRkYR8xJa,Kendrick Lamar,Black Panther The Album Music From And Inspire...,X,1,2,1,4,0.0201,0.768,0.471,0.000000,0.2680,-8.406,0.2590,131.023,0.405,69,hiphop,267.426
spotify:track:5jyyPsIGM2yqkZN9R3TmvN,Kendrick Lamar,Black Panther The Album Music From And Inspire...,The Ways,1,11,0,4,0.0626,0.727,0.720,0.000001,0.1760,-5.856,0.0488,140.080,0.589,65,hiphop,238.893
spotify:track:7bUcBztfGqO7cSI2gMZeCI,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Opps,1,1,1,4,0.1520,0.706,0.775,0.000033,0.4160,-6.819,0.3350,127.929,0.847,59,hiphop,180.893


In [9]:
df_personality_sentiment.head()

,artist,album,name,explicit,genre,chain_lyrics,lyrics,polarity,magnitude,artistic,emotion,imagination,assertive,cheeful,outgoing,modesty,sympathy,fiery,melancholy
0,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Black Panther,1,hiphop,why i go easy \n know why i go easy \n wait \n...,why i go easy know why i go easy wait king of ...,-1.0,1.2,0.999983,0.931594,0.998384,0.095272,0.387043,0.101374,0.421143,0.985013,0.386352,0.921542
1,Kendrick Lamar,Black Panther The Album Music From And Inspire...,All The Stars,1,hiphop,love lets talk about love \n is it anything an...,love lets talk about love is it anything and e...,-1.0,1.3,0.996511,0.986234,0.991554,0.747748,0.986040,0.705842,0.441873,0.842244,0.157679,0.626636
2,Kendrick Lamar,Black Panther The Album Music From And Inspire...,I Am,1,hiphop,everybody put three fingers in the air \n the ...,everybody put three fingers in the air the sky...,-1.0,2.2,0.906971,0.318395,0.994980,0.574279,0.469771,0.320161,0.024560,0.785487,0.582447,0.669157
3,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Big Shot,1,hiphop,wakanda welcome \n big shot hol up wait peanut...,wakanda welcome big shot hol up wait peanut bu...,-1.0,1.3,0.967339,0.229457,0.975867,0.382183,0.880943,0.477181,0.000747,0.270318,0.787414,0.197281
4,Kendrick Lamar,Black Panther The Album Music From And Inspire...,Pray For Me,1,hiphop,im always ready for a war again \n go down tha...,im always ready for a war again go down that r...,-1.0,1.1,0.980110,0.889895,0.977369,0.307116,0.962168,0.668731,0.864149,0.875111,0.464351,0.871338


**I will be trying to predict popularity, so I want popularity in the personality sentiment dataframe.**

In [0]:
df_merge = df_songs[["artist", "name", "album", "popularity"]]
df_personality_sentiment = df_personality_sentiment.merge(
    df_merge, on=["artist", "name", "album"], how="left")

In [11]:
df_personality_sentiment.isnull().sum()

artist          0
album           0
name            0
explicit        0
genre           0
chain_lyrics    0
lyrics          0
polarity        0
magnitude       0
artistic        0
emotion         0
imagination     0
assertive       0
cheeful         0
outgoing        0
modesty         0
sympathy        0
fiery           0
melancholy      0
popularity      0
dtype: int64

**Now I can save these two dataframes.**


In [0]:
df_songs.to_csv("/content/drive/My Drive/df_songs.csv", index=False)

In [0]:
df_personality_sentiment.to_csv(
    "/content/drive/My Drive/df_personality_sentiment.csv", index=False)